# Подготовка данных для mft_db

## Импорт необходимых библиотек

In [39]:
import pandas as pd
import numpy as np
import missingno as msno
import random
import string
import json
import re

from pypinyin import lazy_pinyin

## Сохранение путей к исходным данным

In [97]:
# сохраняем путь к таблице с сырыми данным
mft_rough_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\mft_rough.xlsx'

# сохраняем путь к таблице с готовыми данным
mft_source_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\mft_source.xlsx'

# сохраняем путь к данным о локализации
localization_parts_1_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\localization_parts_1.xlsx'
localization_parts_2_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\localization_parts_2.xlsx'

# сохраняем путь к данные по моделям
model_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\model.xlsx'

# сохраняем путь к данным о конфигурации
configuration_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\configuration.xlsx'

# сохраняем путь к данным BOM
bom_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\bom.xlsx'

# сохраняем путь к данным о производственных линиях
line_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\production_line.xlsx'

# сохраняем путь к данным о производственных цехах
workshop_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\workshop.xlsx'

# сохраняем путь к данным об изменениях номеров деталей
breakpoint_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\sample_breakpoint_data.xlsx'

# сохраняем путь к данным о технических изменениях
bp_list_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\bp_list_2025.xlsx'

# сохраняем путь к json файлу с переводом названий деталей
part_number_translate_path = r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\part_number_translate.json'

## Создание датафреймов с исходными данными

In [42]:
# создаем датафрейм с сырыми данными
mft_rough_df = pd.read_excel(mft_rough_path)

# выводим первые строки датафрейма
mft_rough_df.head()

,cc,партномер,Unnamed: 2,название,Unnamed: 4,A01,A08,B02,B04,B06/B06X,...,ширина_коробки,высота_коробки,V м3 коробки,S м2 коробки,длина_паллеты,ширина_паллеты,высота_паллеты,V м3 паллеты,адрес,штабелирование
0,1,5174104XGW01A,5174104XGW01A,Ниж дефлектор передн бампера в сборе,B02B04B06B16,NaN,NaN,+,+,+,...,560.0,440.0,0.330176,0.7504,1140.0,1480.0,1420.0,2.395824,C05-25-27,1
1,2,5174104XGW01A,5174104XGW01A,Ниж дефлектор передн бампера в сборе,B02B04B06B16,NaN,NaN,+,+,+,...,560.0,440.0,0.330176,0.7504,1140.0,1480.0,1420.0,2.395824,C05-25-27,1
2,3,4004103XKN61A,4004103XKN61A,Жгут проводов салона автомобиля,B02,NaN,NaN,+,NaN,NaN,...,480.0,260.0,0.068640,0.2640,1140.0,1480.0,1140.0,1.923408,Переупаковка-C01-21-28,3
3,4,4004103XKN61A,4004103XKN61A,Жгут проводов салона автомобиля,B02,NaN,NaN,+,NaN,NaN,...,480.0,260.0,0.068640,0.2640,1140.0,1480.0,1140.0,1.923408,Переупаковка-C01-21-28,3
4,5,5174104XGW01A,5174104XGW01A,Ниж дефлектор передн бампера в сборе,B02B04B06B16,NaN,NaN,+,+,+,...,560.0,440.0,0.330176,0.7504,1140.0,1480.0,1420.0,2.395824,C05-25-27,1


In [43]:
# создаем датафрейм с готовыми данными
mft_source_df = pd.read_excel(mft_source_path)

# выводим первые строки датафрейма
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CONFIGURATION,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5.0,NaN,NaN,1340.0,560.0,440.0,NaN,...,NaN,NaN,"Bazhou huihang auto parts co.,Ltd",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5.0,NaN,NaN,1340.0,560.0,440.0,NaN,...,NaN,NaN,"Bazhou huihang auto parts co.,Ltd",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4004103XKN61A,Жгут проводов салона автомобиля,NaN,1.0,NaN,NaN,550.0,480.0,260.0,NaN,...,NaN,NaN,"Mande Electronics Co., Ltd. Baoding Xushui ele...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4004103XKN61A,Жгут проводов салона автомобиля,NaN,1.0,NaN,NaN,550.0,480.0,260.0,NaN,...,NaN,NaN,"Mande Electronics Co., Ltd. Baoding Xushui ele...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5.0,NaN,NaN,1340.0,560.0,440.0,NaN,...,NaN,NaN,"Bazhou huihang auto parts co.,Ltd",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# создаем первый датафрейм с данными по локальным поставщикам
local_df_1 = pd.read_excel(localization_parts_1_path)

# выводим первые строки датафрейма
local_df_1.head()

,PART_NUMBER,PART_NAME,SUPPLIER,PART_PER_CAR,MODEL,WORKSHOP,NAME_CH,STACKING_FACTOR,L,W,H,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,3101101XST33A,车轮总成（18寸）,NDKT,4.0,A01,AS,非折叠无轮工装,4.0,2300.0,1530.0,1330.0,NaN,NaN,NaN
1,3101100XST33A,车轮总成(17寸）,NDKT,NaN,NaN,NaN,非折叠无轮工装,4.0,2300.0,1530.0,1330.0,NaN,NaN,NaN
2,3101100XKN46A,车轮总成（18寸）,NDKT,4.0,B06B06X,AS,非折叠无轮工装,4.0,2300.0,1530.0,1330.0,NaN,NaN,NaN
3,3101102XKN46A,车轮总成（18寸）,NDKT,NaN,NaN,NaN,非折叠无轮工装,4.0,2300.0,1530.0,1330.0,NaN,NaN,NaN
4,3101105XKN46A,备用车轮总成（18寸）,NDKT,1.0,B06,AS,非折叠无轮工装,4.0,2300.0,1530.0,1330.0,NaN,NaN,NaN


In [6]:
# создаем второй датафрейм с данными по локальным поставщикам
local_df_2= pd.read_excel(localization_parts_2_path)

# выводим первые строки датафрейма
local_df_2.head()

,PART_NUMBER,Part Name,SUPPLIER,PART_PER_CAR,MODEL,L,W,H
0,5604108AKN02A,随车工具右侧托板,STP,1.0,B02B04,1200.0,1000.0,2100.0
1,5604107AKN02A,随车工具右侧托板,STP,1.0,B02B04,1200.0,1000.0,2100.0
2,5604102AKN02A,随车工具右侧托板,STP,1.0,B02B04,1200.0,1000.0,2100.0
3,5604101AKN02A,随车工具左侧托板,STP,1.0,B02B04,1200.0,1000.0,2100.0
4,5604101XST11A,工具托板A,Knauf,1.0,A01,1200.0,800.0,1300.0


In [98]:
# создаем датафрейм с данными по моделям
initial_model_df = pd.read_excel(model_path)

# выводим все строки датафрейма
initial_model_df

,MODEL_CODE,MODEL_NAME
0,A01,Jolion
1,A08,H3
2,B02,F7
3,B04,F7x
4,B06,Dargo
5,B16,H7


In [8]:
# создаем датафрейм с данными по конфигурациям
initial_config_df = pd.read_excel(configuration_path)

# выводим первые строки датафрейма
initial_config_df.head()

,PROJECT,PROJECT_CODE,ENGINE_TYPE,DRIVE_TYPE,TRANSMISSION,CONFIGURATION
0,A01,CC7150BA00B,GW4G15K,4X2,MT,comfort
1,A01,CC7150BA00B,GW4G15K,4X2,MT,elite
2,A01,CC7150BA24C,GW4B15D,4x4,AT,elite
3,A01,CC7150BA24C,GW4B15D,4x4,AT,tech plus
4,A01,CC7150BA24C,GW4B15D,4x4,AT,premium


In [32]:
# создаем датафрейм с данными по деталям из бома
initial_bom_df = pd.read_excel(bom_path)

# выводим первые строки датафрейма
initial_bom_df.head()

,MODEL_CODE,PART_NUMBER,LINE_CODE
0,A01,Q1460616FDE,HAA104
1,A01,09810123,HAA117
2,A01,1607011XKR02A,HAA104
3,A01,Q673B135FDE,HAA106
4,A01,09140323,HAC103


In [10]:
# создаем датафрейм с данными по производственным линиям
initial_line_df = pd.read_excel(line_path)

# выводим первые строки датафрейма
initial_line_df.head()

,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME
0,HA0001,Virtual Line,Assembly
1,HAA100,Trim I,Assembly
2,HAA101,Trim I,Assembly
3,HAA102,Trim I,Assembly
4,HAA103,Trim I,Assembly


In [11]:
# создаем датафрейм с данными по производственным цехам
initial_workshop_df = pd.read_excel(workshop_path)

# выводим первые строки датафрейма
initial_workshop_df.head()

,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,Stamping,STAMP
1,Welding,WELD
2,Paint,PAINT
3,Component,COMP
4,Assembly,AS


# Версия 1. Подготовка данных для mft_db

## Обработка данных по поставщикам

### Переименование колонок и удаление мусора из строк с названием поставщиков

In [ ]:
# выбираем нужные колонки для создания рабочего датафрейма
initial_supplier_df = mft_rough_df[['партномер', 'поставщик']].copy()

# выводим первые строки датафрейма
initial_supplier_df.head()

In [ ]:
# переименовываем названия колонок
renamed_supplier_df = initial_supplier_df.rename(columns={
    'партномер': 'PART_NUMBER',
    'поставщик': 'SUPPLIER_NAME'
}, inplace=False)

# выводим первые строки датафрейма
renamed_supplier_df.head()

In [ ]:
# в колонках 'PART_NUMBER' и 'SUPPLIER_NAME' удаляем пробелы в начале и в конце строк
renamed_supplier_df[['PART_NUMBER', 'SUPPLIER_NAME']] = renamed_supplier_df[['PART_NUMBER', 'SUPPLIER_NAME']].apply(lambda x: x.str.strip())

# выводим первые строки датафрейма
renamed_supplier_df.head()

In [ ]:
# функция конвертирования китайский иероглифов в латиницу
def convert_to_latin(row):
    return ''.join(lazy_pinyin(row))

In [ ]:
# в колонке SUPPLIER_NAME конвертируем китайские иероглифы в латиницу
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].apply(convert_to_latin)

# в колонке 'SUPPLIER_NAME' заменяем комбинацию ".," на "_"
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].str.replace('.,', '_', regex=True)

# в колонке 'SUPPLIER_NAME' удаляем точки и запятые
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].str.replace(r'[.,]', '', regex=True)

# в колонке 'SUPPLIER_NAME' заменяем пробелы знаком "_"
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].str.replace(' ', '_', regex=True)

# в колонке 'SUPPLIER_NAME' двойной знак "__" заменяем одинарным "_"
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].str.replace('__', '_', regex=True)

# удаляем символы переноса строки в названиях поставщиков
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].str.replace('\n', '')

# в колонке 'SUPPLIER_NAME' приводим все значения к нижнему регистру
renamed_supplier_df['SUPPLIER_NAME'] = renamed_supplier_df['SUPPLIER_NAME'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# выводим первые строки датафрейма
renamed_supplier_df.head()

### Устранение ошибок в названиях локальных поставщиков

In [ ]:
# выводим строки с вхождением 'ecoall' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('ecoall', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# заменяем строки, содержащих подстроку "ecoall" на 'ecoalliance'
renamed_supplier_df.loc[renamed_supplier_df['SUPPLIER_NAME'].str.contains('ecoall'), 'SUPPLIER_NAME'] = 'ecoalliance'

# выводим результат
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('ecoall', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'ndkt' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('ndkt', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'fuyao' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('fuyao', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'lear' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('lear', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'yapp' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('yapp', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'stp' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('stp', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'gsk' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('gsk', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'sgk' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('sgk', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'aat' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('aat', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'autotec' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('autotec', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'msa' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('msa', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'pure' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('pure', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# заменяем строки, содержащих подстроку "pure" на 'purem_togliatti'
renamed_supplier_df.loc[renamed_supplier_df['SUPPLIER_NAME'].str.contains('pure'), 'SUPPLIER_NAME'] = 'purem_togliatti'

# выводим результат
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('pure', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'knau' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('knau', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'akom' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('akom', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'telm' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('telm', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'ftec' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('ftec', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'smet' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('smet', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# заменяем строки, содержащих подстроку "smet" на 'resmetal'
renamed_supplier_df.loc[renamed_supplier_df['SUPPLIER_NAME'].str.contains('smet'), 'SUPPLIER_NAME'] = 'resmetal'

# выводим результат
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('smet', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'lelastot' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('lelastot', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'vm' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('vm', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'spromh' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('spromh', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'lukoi' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('lukoi', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# заменяем строки, содержащих подстроку "lukoi" на 'lukoil'
renamed_supplier_df.loc[renamed_supplier_df['SUPPLIER_NAME'].str.contains('lukoi'), 'SUPPLIER_NAME'] = 'lukoil'

# выводим результат
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('lukoi', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# выводим строки с вхождением 'litu' названия поставщиков
renamed_supplier_df[renamed_supplier_df['SUPPLIER_NAME'].str.contains('litu', case=False)]['SUPPLIER_NAME'].unique()

In [ ]:
# проверяем дубликаты по колонке PART_NUMBER
renamed_supplier_df[renamed_supplier_df.PART_NUMBER.duplicated()]

In [ ]:
# удаляем дубликаты
drop_duplicates_supplier_df = renamed_supplier_df.drop_duplicates(inplace=False)

# проверяем дубликаты по колонке PART_NUMBER
drop_duplicates_supplier_df[drop_duplicates_supplier_df.PART_NUMBER.duplicated()]

In [ ]:
# проверяем причину дублирования детали ALAB000779
drop_duplicates_supplier_df[drop_duplicates_supplier_df.PART_NUMBER == 'ALAB000779']

In [ ]:
# заменяем название поставщика на gangtuo для детали ALAB000779
drop_duplicates_supplier_df.loc[drop_duplicates_supplier_df['PART_NUMBER'] == 'ALAB000779', 'SUPPLIER_NAME'] = 'gangtuo'

# выводим результат
drop_duplicates_supplier_df[drop_duplicates_supplier_df.PART_NUMBER == 'ALAB000779']

In [ ]:
# удаляем дубликаты
drop_duplicates_supplier_df = drop_duplicates_supplier_df.drop_duplicates().copy()

# проверяем остались ли дубликаты
drop_duplicates_supplier_df[drop_duplicates_supplier_df.PART_NUMBER.duplicated()]

In [ ]:
# создаем копию датафрейма для дальнейшей работы
supplier_to_merge_df = drop_duplicates_supplier_df.copy()

# выводим первые строки датафрейма
supplier_to_merge_df.head()

In [ ]:
# выводим информацию о датафрейме
supplier_to_merge_df.info()

### Извлечение данных о локальных поставщиках

In [ ]:
# выводим названия колонок датафрейма local_df_1
local_df_1.columns

In [ ]:
# удаляем ненужные колонки из датафрейма local_df_1
final_local_df_1 = local_df_1[['PART_NUMBER', 'SUPPLIER']].copy()

# выводим первые строки датафрейма
final_local_df_1.head()

In [ ]:
# выводим названия колонок датафрейма local_df_2
local_df_2.columns

In [ ]:
# удаляем ненужные колонки из датафрейма local_df_2
final_local_df_2 = local_df_2[['PART_NUMBER', 'SUPPLIER']].copy()

# выводим первые строки датафрейма
final_local_df_2.head()

In [ ]:
# выводим размеры датафрейма final_local1_df
print(f'final_local1_df \nКоличество строк: {final_local_df_1.shape[0]}. Количество столбцов: {final_local_df_1.shape[1]}.')

# выводим размеры датафрейма final_local2_df
print(f'final_local2_df \nКоличество строк: {final_local_df_2.shape[0]}. Количество столбцов: {final_local_df_2.shape[1]}.')

In [ ]:
# создаем новый датафрейм local_df из двух,
# объединяя строки из final_local2_df и final_local1_df
local_df = pd.concat([final_local_df_1, final_local_df_2], ignore_index=True)

# выводим размер итогового датафрейма
print(f'Количество строк: {local_df.shape[0]}. Количество столбцов: {local_df.shape[1]}.')

# выводим первые строки датафрейма
local_df.head()

#### Удаление мусора из строк с названием поставщиков

In [ ]:
# в колонке 'SUPPLIER' заменяем пробелы на "_"
local_df['SUPPLIER'] = local_df['SUPPLIER'].str.replace(' ', '_', regex=True)

# значения в колонке SUPPLIER приводим к нижнему регистру
local_df['SUPPLIER'] = local_df['SUPPLIER'].str.lower()

# в колонке SUPPLIER конвертируем китайские иероглифы в латиницу
local_df['SUPPLIER'] = local_df['SUPPLIER'].apply(convert_to_latin)

# выводим первые строки датафрейма
local_df.head()

In [ ]:
# создаем список локальных поставщиков
local_supplier = local_df['SUPPLIER'].drop_duplicates().to_list()
print(local_supplier)

### Добавление информации о локализации в датафрейм о поставщиках

In [ ]:
# выбыраем только локальных поставщиков
local_supplier = [
    'ndkt',
    'fuyao',
    'lear',
    'yapp',
    'gsk',
    'stp',
    'aat',
    'autotechnika',
    'msa',
    'purem_togliatti',
    'knauf',
    'akom',
    'itelma',
    'ecoalliance',
    'litum',
    'resmetal',
    'ecokhim',
    'eftec',
    'lukoil',
    'ruspromholod',
    'sgk',
    'vm_auto',
    'uralelastotehnika',
    'ruichuang'
]

# добавляем колонку LOCALIZATION и ставим значение "yes" для строк,
# где название поставщика совпадает с теми, что в списке, 
# и "no" для тех, где не совпадает
supplier_to_merge_df['LOCALIZATION'] = supplier_to_merge_df['SUPPLIER_NAME'].apply(lambda x: 'yes' if x in local_supplier else 'no')

# выводим первые строки датафрейма
supplier_to_merge_df.head()

In [ ]:
# добавляем новые колонки
supplier_to_merge_df = supplier_to_merge_df.assign(
    SUPPLIER_ID=None,
    LOCATION=None,
    CITY=None,
    STREET=None,
    BUILDING=None
)

# выводим информауию о датафрейме
supplier_to_merge_df.info()

In [ ]:
# фиксируем генератор случайных чисел
random.seed(13)

# определяем функцию для генерации случайного ID поставщиков
def generate_random_id():
    # случайные символы для основной части ID (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем ID с "SUP_" (Supplier) и добавляем случайную часть
    return f"SUP_{random_part}"

In [ ]:
# генерируем уникальные идентификаторы для каждого уникального поставщика
unique_suppliers = supplier_to_merge_df['SUPPLIER_NAME'].unique()
supplier_ids_map = {name: generate_random_id() for name in unique_suppliers}

# присваиваем уникальные ID колонке PART_ID
supplier_to_merge_df['SUPPLIER_ID'] = supplier_to_merge_df['SUPPLIER_NAME'].map(supplier_ids_map)

In [ ]:
# выводим первые строки датафрейма
supplier_to_merge_df.head()

In [ ]:
# выводим информауию о датафрейме
supplier_to_merge_df.info()

## Обработка данных по деталям

### Переименование колонок и удаление мусора и дубликатов из строк с названием деталей

In [ ]:
# выбираем нужные колонки для создания рабочего датафрейма
initial_part_df = mft_rough_df[['партномер', 'название']].copy()

# выводим первые строки датафрейма
initial_part_df.head()

In [ ]:
# переименовываем названия колонок
renamed_part_df = initial_part_df.rename(columns={
    'партномер': 'PART_NUMBER',
    'название': 'PART_NAME'
}, inplace=False)

# выводим первые строки датафрейма
renamed_part_df.head()

In [ ]:
# в колонках 'PART_NUMBER' и 'PART_NAME' удаляем пробелы в начале и в конце строк
renamed_part_df[['PART_NUMBER', 'PART_NAME']] = renamed_part_df[['PART_NUMBER', 'PART_NAME']].apply(lambda x: x.str.strip())

# в колонке 'PART_NAME' заменяем точки на пробел
renamed_part_df['PART_NAME'] = renamed_part_df['PART_NAME'].str.replace(r'[.,]', ' ', regex=True)

# в колонке 'PART_NAME' разделяем слова с помощью знака '_' в строках, которые написаны стилем "CamelCase"
renamed_part_df['PART_NAME'] = renamed_part_df['PART_NAME'].apply(
    lambda x: '_'.join(re.findall(r'[А-Я][^А-Я]*', str(x))) if pd.notna(x) else None
)

# в колонке 'PART_NAME' заменяем пробелы знаком "_"
renamed_part_df['PART_NAME'] = renamed_part_df['PART_NAME'].str.replace(' ', '_', regex=True)

# в колонке 'PART_NAME' двойной знак "__" заменяем одинарным "_"
renamed_part_df['PART_NAME'] = renamed_part_df['PART_NAME'].str.replace('__', '_', regex=True)

# в колонке 'PART_NAME' приводим все значения к нижнему регистру
renamed_part_df['PART_NAME'] = renamed_part_df['PART_NAME'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# в колонке 'PART_NAME' удаляем символы переноса строки
renamed_part_df['PART_NAME'] = renamed_part_df['PART_NAME'].str.replace('\n', '')

# выводим первые строки датафрейма
renamed_part_df.head()

In [ ]:
# проверяем на наличие дубликатов
renamed_part_df[renamed_part_df.duplicated()]

In [ ]:
# удаляем дубликаты
drop_duplicates_part_df = renamed_part_df.drop_duplicates(inplace=False)

# проверяем на наличие дубликатов
drop_duplicates_part_df[drop_duplicates_part_df.duplicated()]

In [ ]:
# добавляем колонку PART_ID
final_part_df = drop_duplicates_part_df.assign(
    PART_ID=None,
    PART_WEIGHT_KG=None
)

# выводим первые строки датафрейма
final_part_df.head()

In [ ]:
# фиксируем генератор случайных чисел
random.seed(23)

# определяем функцию для генерации случайного ID
def generate_random_id():
    # случайные символы для основной части ID (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем ID с "PRT_" (Parts) и добавляем случайную часть
    return f"PRT_{random_part}"

In [ ]:
# генерируем уникальные ID для каждой строки
ids = [generate_random_id() for _ in range(len(final_part_df))]

# присваиваем уникальные ID в колонке PART_ID
final_part_df['PART_ID'] = ids

In [ ]:
# меняем порядок колонок
final_part_df = final_part_df[['PART_ID', 'PART_NUMBER', 'PART_NAME', 'PART_WEIGHT_KG']]

# выводим первые строки датафрейма
final_part_df.head()

In [ ]:
# выводим информацию о датафрейме
final_part_df.info()

## Cохранение данных из результирующего датафрейма по поставщикам

In [ ]:
# удаляем ненужные колонки
final_supplier_df = supplier_to_merge_df.drop(columns='PART_NUMBER', axis=1)

# удаляем дубликаты
final_supplier_df = final_supplier_df.drop_duplicates()

# меняем колонки местами
final_supplier_df = final_supplier_df[
    [
        'SUPPLIER_ID',
        'SUPPLIER_NAME',
        'LOCATION',
        'CITY',
        'STREET',
        'BUILDING', 
        'LOCALIZATION'
    ]
]

# выводим информацию о датафрейме
final_supplier_df.info()

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
supplier = final_supplier_df.to_dict(orient='records')

# cоздаем внешний контейнер "supplier", включив туда все записи о деталях и SUPPLIER_ID
output_data = {"supplier": supplier}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\supplier_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из результирующего датафрейма по деталям

In [ ]:
# объединяем датафреймы по колонке PART_NUMBER
final_part_df = final_part_df.merge(
    supplier_to_merge_df[['PART_NUMBER', 'SUPPLIER_ID']],
    on='PART_NUMBER',
    how='left'
)

# выводим информацию о датафрейме
final_part_df.info()

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
part = final_part_df.to_dict(orient='records')

# cоздаем внешний контейнер "part", включив туда все записи о деталях и SUPPLIER_ID
output_data = {"part": part}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\part_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Обработка данных по моделям

### Создание датафрейма по модельному ряду

In [ ]:
# создаем датафрейм
initial_model_df = pd.read_excel(model_path)

# выводим весь датафрейм
initial_model_df

In [ ]:
# фиксируем генератор случайных чисел
random.seed(33)

# определяем функцию для генерации случайного ID
def generate_random_id():
    # случайные символы для основной части ID (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем ID с "MDL_" (Model) и добавляем случайную часть
    return f"MDL_{random_part}"

In [ ]:
# генерируем уникальные ID для каждой строки
ids = [generate_random_id() for _ in range(len(initial_model_df))]

# присваиваем уникальные идентификаторы колонке MODEL_ID
initial_model_df['MODEL_ID'] = ids

# выводим весь датафрейм
initial_model_df

In [ ]:
# сортируем датафрейм по колонке MODEL_CODE
initial_model_df.sort_values(by='MODEL_CODE').reset_index(drop=True)

# меняем местами колонки
final_model_df = initial_model_df[['MODEL_ID', 'MODEL_NAME', 'MODEL_CODE']]

# выводим весь датафрейм
final_model_df

### Подготовка данных в связующем датафрейме part_to_model

In [ ]:
# выбираем нужные колонки для создания рабочего датафрейма
initial_part_to_model_df = mft_rough_df[
    ['партномер',
     'A01',
     'A08',
     'B02',
     'B04',
     'B06/B06X',
     'B16',
     'конфигурация',
     'кол-во на автомобиль'
    ]
].copy()

# выводим первые строки датафрейма
initial_part_to_model_df.head()

In [ ]:
# переименовываем названия колонок
renamed_part_to_model_df = initial_part_to_model_df.rename(columns={
    'партномер': 'PART_NUMBER',
    'B06/B06X':'B06_B06X',
    'конфигурация': 'CONFIGURATION',
    'кол-во на автомобиль': 'PART_per_VEHICLE'
}, inplace=False)

# выводим первые строки датафрейма
renamed_part_to_model_df.head()

In [ ]:
# преобразовываем колонки CONFIGURATION и PART_per_VEHICLE в Int64
renamed_part_to_model_df[['CONFIGURATION', 'PART_per_VEHICLE']] = renamed_part_to_model_df[
    ['CONFIGURATION', 'PART_per_VEHICLE']
].astype('Int64')

# выводим информацию о датафрейме
renamed_part_to_model_df.info()

In [ ]:
# во всех колонках датафрейма удаляем пробелы в начале и в конце строк
for col in renamed_part_to_model_df.columns:
    if renamed_part_to_model_df[col].dtype == object:
        renamed_part_to_model_df[col] = renamed_part_to_model_df[col].astype(str).str.strip().replace('', np.nan)

# выводим результат
renamed_part_to_model_df.head(10)

In [ ]:
# проверяем на наличие пустых строк
renamed_part_to_model_df[renamed_part_to_model_df.isnull().all(axis=1)]

In [ ]:
# проверяем на наличие дубликатов
renamed_part_to_model_df[renamed_part_to_model_df.duplicated()]

In [ ]:
# удаляем дубликаты
drop_duplicates_part_to_model_df = renamed_part_to_model_df.drop_duplicates(inplace=False)

# проверяем на наличие дубликатов
drop_duplicates_part_to_model_df[drop_duplicates_part_to_model_df.duplicated()]

In [ ]:
# выводим информацию о датафрейме
drop_duplicates_part_to_model_df.info()

#### Добавление данных PART_ID в связующий датафрейм

In [ ]:
# объединяем датафреймы по колонке PART_NUMBER
final_part_to_model_df = drop_duplicates_part_to_model_df.merge(
    final_part_df[['PART_NUMBER', 'PART_ID']],
    on='PART_NUMBER'
)

# выводим первые строки датафрейма
final_part_to_model_df.head()

In [ ]:
# меняем структуру датафрейма final_part_to_model_df
# список столбцов с названиями моделей
model_cols = ['A01', 'A08', 'B02', 'B04', 'B06_B06X', 'B16']

# развертываем таблицу, создавая одну строку на каждую модель, где есть '+'
melted_df = final_part_to_model_df.melt(
    id_vars=['PART_NUMBER', 'CONFIGURATION', 'PART_per_VEHICLE', 'PART_ID'],
    value_vars=model_cols,
    var_name='MODEL_CODE',
    value_name='USED_IN_MODEL'
)

# оставляем только те записи, где используется деталь ('+' в старой таблице)
melted_df = melted_df[melted_df['USED_IN_MODEL'] == '+']

# удаляем ненужный столбец USED_IN_MODEL
melted_df.drop('USED_IN_MODEL', axis=1, inplace=True)

# получаем итоговую версию датафрейма
final_part_to_model_df = melted_df.reset_index(drop=True)

# выводим первые строки датафрейма
final_part_to_model_df.head()

In [ ]:
# объединеняем final_part_to_model_df с final_model_df по колонке PROJECT_CODE
final_part_to_model_df = final_part_to_model_df.merge(
    final_model_df[['MODEL_CODE', 'MODEL_ID']],
    on='MODEL_CODE'
)

# выводим результирующий датафрейм
final_part_to_model_df.head()

In [ ]:
# проверяем датафрейм на дубликаты
final_part_to_model_df[final_part_to_model_df.duplicated()]

In [ ]:
final_part_to_model_df = final_part_to_model_df.drop_duplicates()

# проверяем датафрейм на дубликаты
final_part_to_model_df[final_part_to_model_df.duplicated()]

In [ ]:
# выбираем нужные колонки в нужном порядке для результирующего датафрейма
final_part_to_model_df = final_part_to_model_df[
    [
        'PART_ID',
        'MODEL_ID',
        'CONFIGURATION',
        'PART_per_VEHICLE'
    ]
]

# выводим первые строки датафрейма
final_part_to_model_df.head()

In [ ]:
# выводим информацию о датафрейме
final_part_to_model_df.info()

## Cохранение данных из результирующего датафрейма по моделям

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
model = final_model_df.to_dict(orient='records')

# cоздаем внешний контейнер "model", включив туда все записи о моделях
output_data = {"model": model}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\model_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из связывающего датафрейма part_to_model

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
part_to_model = final_part_to_model_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"part_to_model": part_to_model}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\part_to_model.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Обработка данных по упаковке

### Подготовка общего датафрейма по упаковке

In [ ]:
# создаем рабочий датафрейм
initial_packaging_df = mft_rough_df[
    [
        'кол-во коробок на паллете',
        'длина_коробки',
        'ширина_коробки',
        'высота_коробки',
        'V м3 коробки',
        'S м2 коробки',
        'длина_паллеты',
        'ширина_паллеты',
        'высота_паллеты',
        'V м3 паллеты',
        'штабелирование'
    ]
].copy()

# выводим первые строки датафрейма
initial_packaging_df.head()

In [ ]:
# переименовываем колонки
renamed_packaging_df = initial_packaging_df.rename(
    columns={
        'кол-во коробок на паллете': 'BOX_per_PALLET',
        'длина_коробки': 'BOX_LENGTH_MM',
        'ширина_коробки': 'BOX_WIDTH_MM',
        'высота_коробки': 'BOX_HEIGHT_MM',
        'V м3 коробки': 'BOX_VOL_M3',
        'S м2 коробки': 'BOX_AREA_M2',
        'длина_паллеты': 'PALLET_LENGTH_MM',
        'ширина_паллеты': 'PALLET_WIDTH_MM',
        'высота_паллеты': 'PALLET_HEIGHT_MM',
        'V м3 паллеты': 'PALLET_VOL_M3',
        'штабелирование': 'STATIC_STACKING'
    }, inplace=False
).copy()

# выводим первые строки датафрейма
renamed_packaging_df.head()

In [ ]:
# добавляем новые колонки
renamed_packaging_df = renamed_packaging_df.assign(
    PACKAGING_ID=None,
    PACKAGING_TYPE=None,
    PACKAGING_NUMBER=None,
    PALLET_AREA_M3=None
)

# выводим информацию о датафрейме
renamed_packaging_df.info()

In [ ]:
# строим матрицу датафрейма
msno.matrix(renamed_packaging_df);

In [ ]:
# удаляем данные из колонок BOX_VOL_M3, BOX_AREA_M2, PALLET_VOL_M3, STATIC_STACKING
renamed_packaging_df[['BOX_VOL_M3', 'BOX_AREA_M2', 'PALLET_VOL_M3', 'STATIC_STACKING']] = np.nan

# строим матрицу датафрейма
msno.matrix(renamed_packaging_df);

In [ ]:
# список колонок для преобразования в тип Int64
columns_to_int = [
    'BOX_LENGTH_MM', 'BOX_WIDTH_MM', 'BOX_HEIGHT_MM',
    'PALLET_LENGTH_MM', 'PALLET_WIDTH_MM', 'PALLET_HEIGHT_MM'
]

# для каждой колонки из списка преобразуем значения в тип Int64
for col in columns_to_int:
    renamed_packaging_df[col] = renamed_packaging_df[col].astype('Int64')

# выводим информацию о датафрейме
renamed_packaging_df.info()

In [ ]:
# проверяем наличие пустых строк в датафрейме
renamed_packaging_df[renamed_packaging_df.isnull().all(axis=1)]

In [ ]:
# удаляем только те строки, в которых все ячейки содержат пустые значения,
# если хотя бы одно значение присутствует, строка останется
renamed_packaging_df.dropna(how='all', inplace=True)

# проверяем наличие пустых строк в датафрейме
renamed_packaging_df[renamed_packaging_df.isnull().all(axis=1)]

In [ ]:
# если в колонках BOX_*_MM нет данных 
# копируем в них данные из колонок PALLET_*_MM

# маска для проверки наличия пустых значений в колонках BOX_*_MM
mask = (
    (renamed_packaging_df['BOX_LENGTH_MM'].isna()) &
    (renamed_packaging_df['BOX_WIDTH_MM'].isna()) &
    (renamed_packaging_df['BOX_HEIGHT_MM'].isna())
)

# копируем значения из PALLET_*_MM в BOX_*_MM там, где условие выполняется
renamed_packaging_df.loc[mask, 'BOX_LENGTH_MM'] = renamed_packaging_df.loc[mask, 'PALLET_LENGTH_MM']
renamed_packaging_df.loc[mask, 'BOX_WIDTH_MM'] = renamed_packaging_df.loc[mask, 'PALLET_WIDTH_MM']
renamed_packaging_df.loc[mask, 'BOX_HEIGHT_MM'] = renamed_packaging_df.loc[mask, 'PALLET_HEIGHT_MM']

# устанавливаем значение 1 в колонку BOX_per_PALLET,
# так как предполагается, что на 1 паллету ставится 1 ящик
renamed_packaging_df.loc[mask, 'BOX_per_PALLET'] = 1

# выводим информацию о датафрейме
renamed_packaging_df.info()

In [ ]:
# проверяем наличие дублирующихся строк
renamed_packaging_df[renamed_packaging_df.duplicated()]

In [ ]:
# удаляем дублирующиеся строки
renamed_packaging_df = renamed_packaging_df.drop_duplicates()

# проверяем наличие дублирующихся строк
renamed_packaging_df[renamed_packaging_df.duplicated()]

In [ ]:
# проверяем есть ли значения в колонке BOX_LENGTH_MM больше чем в колонке PALLET_LENGTH_MM, 
# но меньше чем в колонке PALLET_WIDTH_MM
renamed_packaging_df[
    (renamed_packaging_df['BOX_LENGTH_MM'] > renamed_packaging_df['PALLET_LENGTH_MM']) & 
    (renamed_packaging_df['BOX_LENGTH_MM'] < renamed_packaging_df['PALLET_WIDTH_MM'])
]

In [ ]:
# маска для хранения данных, где длина коробки больше чем длина поддона, 
# но меньше чем ширина поддона
mask = (
    (renamed_packaging_df['BOX_LENGTH_MM'] > renamed_packaging_df['PALLET_LENGTH_MM']) &
    (renamed_packaging_df['BOX_LENGTH_MM'] < renamed_packaging_df['PALLET_WIDTH_MM'])
)

# сохраняем временные переменные для значений ширины и длины
temp_length = renamed_packaging_df.loc[mask, 'PALLET_LENGTH_MM']
temp_width = renamed_packaging_df.loc[mask, 'PALLET_WIDTH_MM']

# меняем местами значения
renamed_packaging_df.loc[mask, 'PALLET_LENGTH_MM'] = temp_width
renamed_packaging_df.loc[mask, 'PALLET_WIDTH_MM'] = temp_length

# проверяем остались ли такие строки, где длина коробки больше чем длина поддона, 
# но меньше чем ширина поддона
renamed_packaging_df[
    (renamed_packaging_df['BOX_LENGTH_MM'] > renamed_packaging_df['PALLET_LENGTH_MM']) & 
    (renamed_packaging_df['BOX_LENGTH_MM'] < renamed_packaging_df['PALLET_WIDTH_MM'])
]

In [ ]:
# выводим информацию о датафрейме
renamed_packaging_df.info()

### Разбиение общего датафрейма renamed_packaging_df на в box_df, part_to_box_df, pallet_df, box_to_pallet_df

In [ ]:
# делим renamed_packaging_df на box_df, pallet_df, box_to_pallet_df
# колонки для box_df
box_columns = renamed_packaging_df.columns.intersection([
    'PACKAGING_ID', 
    'PACKAGING_TYPE', 
    'PACKAGING_NUMBER', 
    'BOX_LENGTH_MM',
    'BOX_WIDTH_MM',
    'BOX_HEIGHT_MM',
    'BOX_VOL_M3',
    'BOX_AREA_M2',
    'STATIC_STACKING'
])

# колонки для pallet_df
pallet_columns = renamed_packaging_df.columns.intersection([
    'PACKAGING_ID', 
    'PACKAGING_TYPE', 
    'PACKAGING_NUMBER', 
    'PALLET_LENGTH_MM',
    'PALLET_WIDTH_MM',
    'PALLET_HEIGHT_MM',
    'PALLET_VOL_M3',
    'PALLET_AREA_M2',
    'STATIC_STACKING'
])

# создаем новые датафреймы
box_df = renamed_packaging_df[box_columns].reset_index(drop=True).copy()
pallet_df = renamed_packaging_df[pallet_columns].reset_index(drop=True).copy()

### Обработка данных по ящикам

In [ ]:
# создаем копию датафрейма, чтоб не производить изменения inplace
final_box_df = box_df.copy()

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# переименовываем колонки в датафрейме final_box_df
final_box_df.rename(
    columns={
        'PACKAGING_ID': 'BOX_ID',
        'PACKAGING_TYPE': 'BOX_TYPE',
        'PACKAGING_NUMBER': 'BOX_NUMBER',
        'STATIC_STACKING': 'BOX_STACKING'
    },
    inplace=True
)

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# проверяем наличие пустых строк в датафрейме
final_box_df[final_box_df.isnull().all(axis=1)]

In [ ]:
# проверяем датафрейм на наличие дубликатов
final_box_df[final_box_df.duplicated()]

In [ ]:
# удаляем дублирующиеся строки
final_box_df = final_box_df.drop_duplicates()

# проверяем на наличие дубликатов
final_box_df[final_box_df.duplicated()]

In [ ]:
# пересчитываем объем коробок в датафрейме box_df
final_box_df['BOX_VOL_M3'] = round(
    (box_df['BOX_LENGTH_MM'] / 1000) * 
    (box_df['BOX_WIDTH_MM'] / 1000) * 
    (box_df['BOX_HEIGHT_MM'] / 1000), 2)

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# пересчитываем площадь коробок в датафрейме box_df
final_box_df['BOX_AREA_M2'] = round(
    (final_box_df['BOX_LENGTH_MM'] / 1000) * 
    (final_box_df['BOX_WIDTH_MM'] / 1000), 2)

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# создаем в колонке BOX_NUMBER номера коробок,
# сцепляя значения из колонок BOX_LENGTH_MM, BOX_WIDTH_MM, BOX_HEIGHT_MM,
# присваивая в начале номера литеру "А" - "невозвратная тара"

final_box_df['BOX_NUMBER'] = final_box_df.apply(
    lambda row: f"A {row['BOX_LENGTH_MM']}-{row['BOX_WIDTH_MM']}-{row['BOX_HEIGHT_MM']}",
    axis=1
)

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# вносим значение 0 для всей колонки BOX_STACKING
final_box_df['BOX_STACKING'] = 0

# преобразовываем колонки BOX_STACKING в Int64
final_box_df['BOX_STACKING'] = final_box_df['BOX_STACKING'].astype('Int64')

# выводим информацию о датафрейме
final_box_df.info()

In [ ]:
# вносим значение n_returnable (невозвратная) в колонку BOX_TYPE
final_box_df['BOX_TYPE'] = 'n_returnable'

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# фиксируем генератор случайных чисел
random.seed(43)

# определяем функцию для генерации случайного идентификатора
def generate_random_id():
    # случайные символы для основной части идентификатора (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем идентификатор с "BX_" (Boxes) и добавляем случайную часть
    return f"BX_{random_part}"

In [ ]:
# генерируем уникальные идентификаторы для каждой строки
ids = [generate_random_id() for _ in range(len(final_box_df))]

# присваиваем уникальные идентификаторы колонке BOX_ID
final_box_df['BOX_ID'] = ids

In [ ]:
# меняем колонки местами
final_box_df = final_box_df[
    [
        'BOX_ID',
        'BOX_NUMBER',
        'BOX_TYPE',
        'BOX_LENGTH_MM',
        'BOX_WIDTH_MM',
        'BOX_HEIGHT_MM',
        'BOX_VOL_M3',
        'BOX_AREA_M2',
        'BOX_STACKING'
    ]
]

# выводим первые строки датафрейма
final_box_df.head()

In [ ]:
# выводим информацию о датафрейме
final_box_df.info()

### Обработка данных в связующем датафрейме part_to_box_df

In [ ]:
# переименовываем колонки
part_to_box_df = mft_rough_df.rename(
    columns={
        'партномер': 'PART_NUMBER',
        'кол-во деталей в коробке': 'PART_per_BOX',
        'длина_коробки': 'BOX_LENGTH_MM',
        'ширина_коробки': 'BOX_WIDTH_MM',
        'высота_коробки': 'BOX_HEIGHT_MM',
        'длина_паллеты': 'PALLET_LENGTH_MM',
        'ширина_паллеты': 'PALLET_WIDTH_MM',
        'высота_паллеты': 'PALLET_HEIGHT_MM'
    }
)

# оcтавляем в датафрейме только переименованные колонки
part_to_box_df = part_to_box_df[
    [
        'PART_NUMBER',
        'PART_per_BOX',
        'BOX_LENGTH_MM',
        'BOX_WIDTH_MM',
        'BOX_HEIGHT_MM',
        'PALLET_LENGTH_MM',
        'PALLET_WIDTH_MM',
        'PALLET_HEIGHT_MM'
    ]
]

# выводим информацию о датафрейме
part_to_box_df.info()

In [ ]:
# если длина коробки больше чем длина поддона, но меньше чем ширина поддона,
# меняем местами значения PALLET_LENGTH_MM и PALLET_WIDTH_MM

# маска для хранения данных, где длина коробки больше чем длина поддона, но меньше чем ширина поддона
mask = (
    (part_to_box_df['BOX_LENGTH_MM'] > part_to_box_df['PALLET_LENGTH_MM']) &
    (part_to_box_df['BOX_LENGTH_MM'] < part_to_box_df['PALLET_WIDTH_MM'])
)

# сохраняем временные переменные для значений ширины и длины
temp_length = part_to_box_df.loc[mask, 'PALLET_LENGTH_MM']
temp_width = part_to_box_df.loc[mask, 'PALLET_WIDTH_MM']

# меняем местами значения
part_to_box_df.loc[mask, 'PALLET_LENGTH_MM'] = temp_width
part_to_box_df.loc[mask, 'PALLET_WIDTH_MM'] = temp_length

In [ ]:
# если в колонках BOX_*_MM нет данных 
# копируем в них данные из колонок PALLET_*_MM

# маска для проверки наличие пустых значений в колонках BOX_*_MM
mask = (
    (part_to_box_df['BOX_LENGTH_MM'].isna()) &
    (part_to_box_df['BOX_WIDTH_MM'].isna()) &
    (part_to_box_df['BOX_HEIGHT_MM'].isna())
)

# копируем значения из PALLET_*_MM в BOX_*_MM там, где условие выполняется
part_to_box_df.loc[mask, 'BOX_LENGTH_MM'] = part_to_box_df.loc[mask, 'PALLET_LENGTH_MM']
part_to_box_df.loc[mask, 'BOX_WIDTH_MM'] = part_to_box_df.loc[mask, 'PALLET_WIDTH_MM']
part_to_box_df.loc[mask, 'BOX_HEIGHT_MM'] = part_to_box_df.loc[mask, 'PALLET_HEIGHT_MM']

# выводим информацию о датафрейме
part_to_box_df.info()

In [ ]:
# список колонок для преобразования в тип Int64
columns_to_int = [
    'PART_per_BOX',
    'BOX_LENGTH_MM',
    'BOX_WIDTH_MM',
    'BOX_HEIGHT_MM',
    'PALLET_LENGTH_MM',
    'PALLET_WIDTH_MM',
    'PALLET_HEIGHT_MM'
]

# для каждой колонки из списка преобразуем значения в тип Int64
for col in columns_to_int:
    part_to_box_df[col] = pd.to_numeric(part_to_box_df[col], errors='coerce').fillna(np.nan).astype('Int64')

# выводим информацию о датафрейме
part_to_box_df.info()

In [ ]:
# создаем колонку BOX_NUMBER соединяя длину, ширину и высоту коробки
part_to_box_df['BOX_NUMBER'] = part_to_box_df.apply(
    lambda row: f"A {row['BOX_LENGTH_MM']}-{row['BOX_WIDTH_MM']}-{row['BOX_HEIGHT_MM']}",
    axis=1
)

# выводим первые строки датафрейма
part_to_box_df.head()

In [ ]:
# удаляем ненужные колонки
final_part_to_box_df = part_to_box_df.drop(
    columns=[
        'BOX_LENGTH_MM',
        'BOX_WIDTH_MM',
        'BOX_HEIGHT_MM',
        'PALLET_LENGTH_MM',
        'PALLET_WIDTH_MM',
        'PALLET_HEIGHT_MM'
    ], inplace=False
).copy()

# удаляем пробелы в начале и в конце значений колонки PART_NUMBER
final_part_to_box_df['PART_NUMBER'] = final_part_to_box_df['PART_NUMBER'].str.strip()

# выводим информацию о датафрейме
final_part_to_box_df.info()

In [ ]:
# удаляем дубликаты
final_part_to_box_df = final_part_to_box_df.drop_duplicates()

# выводим информацию о датафрейме
final_part_to_box_df.info()

In [ ]:
# проверяем на налиние значений 'A <NA>-<NA>-<NA>' в колонке BOX_NUMBER
final_part_to_box_df.loc[final_part_to_box_df['BOX_NUMBER'] == 'A <NA>-<NA>-<NA>']

In [ ]:
# заменяем значения 'A <NA>-<NA>-<NA>' из колонки BOX_NUMBER на NA
final_part_to_box_df.loc[final_part_to_box_df['BOX_NUMBER'] == 'A <NA>-<NA>-<NA>', 'BOX_NUMBER'] = pd.NA

# выводим информацию о датафрейме
final_part_to_box_df.info()

In [ ]:
# вносим в final_part_to_box_df информацию PART_ID из final_part_df
final_part_to_box_df = final_part_to_box_df.merge(
    final_part_df[['PART_NUMBER', 'PART_ID']],
    on='PART_NUMBER'
)

# выводим информацию о датафрейме
final_part_to_box_df.info()

In [ ]:
# вносим в final_part_to_box_df информацию BOX_ID из final_box_df
final_part_to_box_df = final_part_to_box_df.merge(
    final_box_df[['BOX_NUMBER', 'BOX_ID']],
    on='BOX_NUMBER'
)

# выводим информацию о датафрейме
final_part_to_box_df.info()

In [ ]:
# сохраняем в результирующий датафрейм нужные колонки в нужном порядке
final_part_to_box_df = final_part_to_box_df[['PART_ID', 'BOX_ID', 'PART_per_BOX']]

# выводим первые строки датафрейма
final_part_to_box_df.head()

In [ ]:
# выводим информацию о датафрейме
final_part_to_box_df.info()

In [ ]:
# проверяем датафрейм на наличие дубликатов по двум колонкам PART_ID и BOX_ID
final_part_to_box_df[final_part_to_box_df[['PART_ID', 'BOX_ID']].duplicated()]

In [ ]:
# выбираем строки с максимальным значением PART_per_BOX по двум колонкам PART_ID и BOX_ID
final_part_to_box_df = final_part_to_box_df.groupby(['PART_ID', 'BOX_ID'])['PART_per_BOX'].max().reset_index()

# снова проверяем датафрейм на наличие дубликатов по двум колонкам PART_ID и BOX_ID
final_part_to_box_df[final_part_to_box_df.duplicated()]

In [ ]:
# выводим информацию о датафрейме
final_part_to_box_df.info()

### Обработка данных по паллетам

In [ ]:
# создаем копию датафрейма, чтоб не производить изменения inplace
final_pallet_df = pallet_df.copy()

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# переименовываем колонки
final_pallet_df.rename(
    columns={
        'PACKAGING_ID': 'PALLET_ID',
        'PACKAGING_TYPE': 'PALLET_TYPE',
        'PACKAGING_NUMBER': 'PALLET_NUMBER',
        'STATIC_STACKING': 'PALLET_STACKING'
    }, 
    inplace=True
)

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# проверяем наличие пустых строк в датафрейме
final_pallet_df[final_pallet_df.isnull().all(axis=1)]

In [ ]:
# удаляем только те строки, в которых все ячейки содержат пустые значения
# если хотя бы одно значение присутствует, строка останется
final_pallet_df.dropna(how='all', inplace=True)

# снова проверяем наличие пустых строк в датафрейме
final_pallet_df[final_pallet_df.isnull().all(axis=1)]

In [ ]:
# проверяем датафрейм на наличие дубликатов
final_pallet_df[final_pallet_df.duplicated()]

In [ ]:
# удаляем дублирующиеся строки
final_pallet_df = final_pallet_df.drop_duplicates()

# снова проверяем на наличие дубликатов
final_pallet_df[final_pallet_df.duplicated()]

In [ ]:
# пересчитываем объем паллет в датафрейме pallet_df
final_pallet_df['PALLET_VOL_M3'] = round(
    (final_pallet_df['PALLET_LENGTH_MM'] / 1000) * 
    (final_pallet_df['PALLET_WIDTH_MM'] / 1000) * 
    (final_pallet_df['PALLET_HEIGHT_MM'] / 1000), 2)

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# пересчитываем площадь паллет в датафрейме pallet_df
final_pallet_df['PALLET_AREA_M2'] = round(
    (final_pallet_df['PALLET_LENGTH_MM'] / 1000) * 
    (final_pallet_df['PALLET_WIDTH_MM'] / 1000), 2)

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# создаем колонку PALLET_NUMBER соединяя длину, ширину и высоту коробки
final_pallet_df['PALLET_NUMBER'] = final_pallet_df.apply(
    lambda row: f"A {int(row['PALLET_LENGTH_MM'])}-{int(row['PALLET_WIDTH_MM'])}-{int(row['PALLET_HEIGHT_MM'])}",
    axis=1
)

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# вносим значение 0 для всей колонки PALLET_STACKING
final_pallet_df['PALLET_STACKING'] = 0

# преобразовываем колонки PALLET_STACKING в Int64
final_pallet_df['PALLET_STACKING'] = final_pallet_df['PALLET_STACKING'].astype('Int64')

# выводим информацию о датафрейме
final_pallet_df.info()

In [ ]:
# вносим значение n_returnable (невозвратная) для всей колонки PALLET_TYPE
final_pallet_df['PALLET_TYPE'] = 'n_returnable'

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# фиксируем генератор случайных чисел
random.seed(53)

# определяем функцию для генерации случайного идентификатора
def generate_random_id():
    # случайные символы для основной части идентификатора (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем идентификатор с "PLT_" (Pallets) и добавляем случайную часть
    return f"PLT_{random_part}"

In [ ]:
# генерируем уникальные идентификаторы для каждой строки
ids = [generate_random_id() for _ in range(len(final_pallet_df))]

# присваиваем уникальные идентификаторы колонке PALLET_ID
final_pallet_df['PALLET_ID'] = ids

# выводим первые строки датафрейма
final_pallet_df.head()

In [ ]:
# меняем колонки местами
final_pallet_df = final_pallet_df[
    [
        'PALLET_ID',
        'PALLET_NUMBER',
        'PALLET_TYPE',
        'PALLET_LENGTH_MM',
        'PALLET_WIDTH_MM',
        'PALLET_HEIGHT_MM',
        'PALLET_VOL_M3',
        'PALLET_AREA_M2',
        'PALLET_STACKING'
    ]
]

# выводим информацию о датафрейме
final_pallet_df.info()

### Обработка данных в связующем датафрейме box_to_pallet_df

In [ ]:
# проверка уникальности комбинаций размеров коробок
box_duplicates = final_box_df.duplicated(subset=['BOX_LENGTH_MM', 'BOX_WIDTH_MM', 'BOX_HEIGHT_MM'])
if box_duplicates.any():
    print("Есть повторяющиеся комбинации размеров коробок.")
else:
    print("Комбинации размеров коробок уникальны.")

# проверка уникальности комбинаций размеров паллет
pallet_duplicates = final_pallet_df.duplicated(subset=['PALLET_LENGTH_MM', 'PALLET_WIDTH_MM', 'PALLET_HEIGHT_MM'])
if pallet_duplicates.any():
    print("Есть повторяющиеся комбинации размеров паллет.")
else:
    print("Комбинации размеров паллет уникальны.")

In [ ]:
# удаляем ненужные колонки
box_to_pallet_df = renamed_packaging_df.drop(
    columns=[
        'BOX_VOL_M3',
        'BOX_AREA_M2',
        'PALLET_VOL_M3',
        'STATIC_STACKING',
        'PACKAGING_ID',
        'PACKAGING_TYPE',
        'PACKAGING_NUMBER',
        'PALLET_AREA_M3'
    ], axis=1, inplace=False
)

# выводим первые строки датафрейма
box_to_pallet_df.head()

In [ ]:
# соединяем box_to_pallet_df с final_box_df для получения BOX_ID
box_to_pallet_df = box_to_pallet_df.merge(
    final_box_df[['BOX_LENGTH_MM', 'BOX_WIDTH_MM', 'BOX_HEIGHT_MM', 'BOX_ID']],
    on=['BOX_LENGTH_MM', 'BOX_WIDTH_MM', 'BOX_HEIGHT_MM'],
    how='left'
)

# соединяем box_to_pallet_df с final_pallet_df для получения PALLET_ID
box_to_pallet_df = box_to_pallet_df.merge(
    final_pallet_df[['PALLET_LENGTH_MM', 'PALLET_WIDTH_MM', 'PALLET_HEIGHT_MM', 'PALLET_ID']],
    on=['PALLET_LENGTH_MM', 'PALLET_WIDTH_MM', 'PALLET_HEIGHT_MM'],
    how='left'
)

# выводим первые строки датафрейма
box_to_pallet_df.head()

In [ ]:
# выбираем нужные колонки для final_box_to_pallet_df
final_box_to_pallet_df = box_to_pallet_df[
    [
        'BOX_ID',
        'PALLET_ID',
        'BOX_per_PALLET'
    ]
]

# выводим информацию о датафрейме
final_box_to_pallet_df.info()

In [ ]:
# преобразовываем колонку BOX_per_PALLET в Int64
final_box_to_pallet_df['BOX_per_PALLET'] = final_box_to_pallet_df['BOX_per_PALLET'].astype('Int64')

# выводим информацию о датафрейме
final_box_to_pallet_df.info()

In [ ]:
# выводим строки, где не установлен BOX_ID
final_box_to_pallet_df[final_box_to_pallet_df['BOX_ID'].isna()]

In [ ]:
# выводим строки, где не установлен PALLET_ID
final_box_to_pallet_df[final_box_to_pallet_df['PALLET_ID'].isna()]

In [ ]:
# удаляем строки, где не установлен PALLET_ID
final_box_to_pallet_df.dropna(subset=['PALLET_ID'], inplace=True)

In [ ]:
# выводим информацию о датафрейме
final_box_to_pallet_df.info()

In [ ]:
# проверяем датафрейм на наличие дубликатов по двум колонкам PART_ID и BOX_ID
final_box_to_pallet_df[final_box_to_pallet_df[['BOX_ID', 'PALLET_ID']].duplicated()]

In [ ]:
# выбираем максимальное значение для дубликатов по двум колонкам PART_ID и BOX_ID
final_box_to_pallet_df = final_box_to_pallet_df.groupby(['BOX_ID', 'PALLET_ID'])['BOX_per_PALLET'].max().reset_index()

# снова проверяем датафрейм на наличие дубликатов по двум колонкам PART_ID и BOX_ID
final_box_to_pallet_df[final_box_to_pallet_df.duplicated()]

In [ ]:
# выводим первые строки датафрейма
final_box_to_pallet_df.head()

In [ ]:
# выводим информацию о датафрейме
final_box_to_pallet_df.info()

## Cохранение данных из результирующего датафрейма по ящиками

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
box = final_box_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"box": box}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\box_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из связующего датафрейма part_to_box_df

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
part_to_box = final_part_to_box_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"part_to_box": part_to_box}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\part_to_box.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из результирующего датафрейма по паллетам

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
pallet = final_pallet_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"pallet": pallet}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\pallet_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из связующего датафрейма box_to_pallet_df

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
box_to_pallet = final_box_to_pallet_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"box_to_pallet": box_to_pallet}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\box_to_pallet.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Обработка данных по производственным цехам и линиям

### Обработка данных по производственным цехам

In [ ]:
# выводим первые строки датафрейма
initial_workshop_df.head()

In [ ]:
# фиксируем генератор случайных чисел
random.seed(63)

# определяем функцию для генерации случайного идентификатора
def generate_random_id():
    # случайные символы для основной части идентификатора (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем идентификатор с "WS_" (Workshop) и добавляем случайную часть
    return f"WS_{random_part}"

In [ ]:
# генерируем уникальные идентификаторы для каждой строки
ids = [generate_random_id() for _ in range(len(initial_workshop_df))]

# присваиваем уникальные идентификаторы колонке PALLET_ID
initial_workshop_df['WORKSHOP_ID'] = ids

# создаем копию датафрейма
final_workshop_df = initial_workshop_df.copy()

# выводим первые строки датафрейма
final_workshop_df.head()

### Обработка данных по производственным линиям

In [ ]:
# выводим первые строки датафрейма
initial_line_df.head()

In [ ]:
# проверяем датафрейм на дубликаты
initial_line_df[initial_line_df.duplicated()]

In [ ]:
# фиксируем генератор случайных чисел
random.seed(73)

# определяем функцию для генерации случайного идентификатора
def generate_random_id():
    # случайные символы для основной части идентификатора (6 символов)
    random_part = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    
    # начинаем идентификатор с "LN_" (Line) и добавляем случайную часть
    return f"LN_{random_part}"

In [ ]:
# генерируем уникальные идентификаторы для каждой строки
ids = [generate_random_id() for _ in range(len(initial_line_df))]

# присваиваем уникальные идентификаторы колонке PALLET_ID
initial_line_df['LINE_ID'] = ids

# выводим первые строки датафрейма
initial_line_df.head()

In [ ]:
# объединяем датафреймы по колонке WORKSHOP_FULL_NAME
initial_line_df = initial_line_df.merge(
    initial_workshop_df[['WORKSHOP_FULL_NAME', 'WORKSHOP_ID']],
    on='WORKSHOP_FULL_NAME',
    how='left'
)

# выводим первые строки датафрейма
initial_line_df.head()

In [ ]:
# выбираем нужные колонки датафрейма
final_line_df = initial_line_df[
    [
        'LINE_ID',
        'LINE_CODE',
        'LINE_NAME',
        'WORKSHOP_ID'
    ]
]

# выводим первые строки датафрейма
final_line_df.head()

In [ ]:
# выводим информацию о датафрейме
final_line_df.info()

### Обработка данных в связующем датафрейме part_to_line_df

In [ ]:
# выводим первые строки датафрейма
initial_bom_df.head()

In [ ]:
# переименовываем колонки
initial_bom_df.rename(
    columns={
        'model': 'MODEL_CODE',
        'part_number': 'PART_NUMBER',
        'workshop': 'LINE_CODE',
    }, 
    inplace=True
)

# выводим первые строки датафрейма
initial_bom_df.head()

In [ ]:
# выводим информацию о датафрейме
initial_bom_df.info()

In [ ]:
# объединяем датафреймы по колонке LINE_CODE
initial_bom_df = initial_bom_df.merge(
    initial_line_df[['LINE_CODE', 'LINE_ID']],
    on='LINE_CODE',
    how='left'
)

# выводим первые строки датафрейма
initial_bom_df.head()

In [ ]:
# выводим информацию о датафрейме
initial_bom_df.info()

In [ ]:
# объединяем датафреймы по колонке PART_NUMBER
initial_bom_df = initial_bom_df.merge(
    final_part_df[['PART_NUMBER', 'PART_ID']],
    on='PART_NUMBER',
    how='left'
)

# выводим первые строки датафрейма
initial_bom_df.head()

In [ ]:
# выводим информацию о датафрейме
initial_bom_df.info()

In [ ]:
# выбираем нужные колонки из датафрейма
part_to_line_df = initial_bom_df[
    [
        'PART_ID',
        'LINE_ID'
    ]
]

# выводим первые строки датафрейма
part_to_line_df.head()

In [ ]:
# проверяем датафрейм на дубликаты
part_to_line_df[part_to_line_df.duplicated()]

In [ ]:
# удаляем дубликаты
final_part_to_line_df = part_to_line_df.drop_duplicates()

# снова проверяем датафрейм на дубликаты
final_part_to_line_df[final_part_to_line_df.duplicated()]

In [ ]:
# проверяем датафрейм на пропуски в колонке PART_ID
final_part_to_line_df[final_part_to_line_df['PART_ID'].isna()]

In [ ]:
# удаляем строки с пропусками в колонке PART_ID
final_part_to_line_df = final_part_to_line_df.dropna(subset=['PART_ID'])

# снова проверяем датафрейм на пропуски в колонке PART_ID
final_part_to_line_df[final_part_to_line_df['PART_ID'].isna()]

In [ ]:
# проверяем датафрейм на пропуски в колонке LINE_ID
final_part_to_line_df[final_part_to_line_df['LINE_ID'].isna()]

In [ ]:
# удаляем строки с пропусками в колонке LINE_ID
final_part_to_line_df = final_part_to_line_df.dropna(subset=['LINE_ID'])

# снова проверяем датафрейм на пропуски в колонке LINE_ID
final_part_to_line_df[final_part_to_line_df['LINE_ID'].isna()]

In [ ]:
# выводим информацию о датафрейме
final_part_to_line_df.info()

## Cохранение данных из результирующего датафрейма по производственным цехам

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
workshop = final_workshop_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"workshop": workshop}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\workshop_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из результирующего датафрейма по производственным линиям

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
line = final_line_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"line": line}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\line_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

## Cохранение данных из связующего датафрейма part_to_line_df

In [ ]:
# преобразовываем DataFrame в список словарей (каждая строка — отдельное представление)
part_to_line = final_part_to_line_df.to_dict(orient='records')

# cоздаем внешний контейнер "part_to_model", включив туда все связующие записи о PART_ID, MODEL_ID, 'CONFIGURATION', 'PART_per_VEHICLE'
output_data = {"part_to_line": part_to_line}

# записываем итоговую структуру в JSON-файл
with open(r'..\data\part_to_line.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

# Версия 2. Подготовка данных для mft_db

In [12]:
# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22297 entries, 0 to 22296
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          22297 non-null  object 
 1   PART_NAME            22292 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         15143 non-null  float64
 4   BOX_NUMBER           0 non-null      float64
 5   BOX_TYPE             0 non-null      float64
 6   BOX_LENGTH_MM        15337 non-null  float64
 7   BOX_WIDTH_MM         15337 non-null  float64
 8   BOX_HEIGHT_MM        15337 non-null  float64
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      float64
 12  BOX_per_PALLET       15146 non-null  float64
 13  LINE_CODE            0 non-null      float64
 14  LINE_NAME            0 non-null      float64
 15  MODEL_NAME           0 non-null     

In [13]:
# колонки для преобразования в тип str
columns_to_str = [
    'MODEL_CODE',
    'MODEL_NAME',
    'LINE_CODE',
    'LINE_NAME',
    'WORKSHOP_full_NAME',
    'WORKSHOP_short_NAME',
    'BOX_NUMBER',
    'BOX_TYPE',
    'PALLET_NUMBER',
    'PALLET_TYPE', 
    'SUPPLIER_NAME',
    'LOCATION',
    'CITY',
    'STREET',
    'BUILDING',
    'LOCALIZATION'
]

# преобразовываем колонки в тип str
mft_source_df[columns_to_str] = mft_source_df[columns_to_str].astype(str)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22297 entries, 0 to 22296
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          22297 non-null  object 
 1   PART_NAME            22292 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         15143 non-null  float64
 4   BOX_NUMBER           22297 non-null  object 
 5   BOX_TYPE             22297 non-null  object 
 6   BOX_LENGTH_MM        15337 non-null  float64
 7   BOX_WIDTH_MM         15337 non-null  float64
 8   BOX_HEIGHT_MM        15337 non-null  float64
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      float64
 12  BOX_per_PALLET       15146 non-null  float64
 13  LINE_CODE            22297 non-null  object 
 14  LINE_NAME            22297 non-null  object 
 15  MODEL_NAME           22297 non-null 

In [14]:
columns_to_int = [
    'PART_per_VEHICLE',
    'PART_per_BOX',
    'BOX_LENGTH_MM',
    'BOX_WIDTH_MM',
    'BOX_HEIGHT_MM',
    'BOX_STACKING',
    'BOX_per_PALLET',
    'PALLET_LENGTH_MM',
    'PALLET_WIDTH_MM',
    'PALLET_HEIGHT_MM',
    'PALLET_STACKING'
]

# преобразовываем колонки в тип str
mft_source_df[columns_to_int] = mft_source_df[columns_to_int].astype('Int64')

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22297 entries, 0 to 22296
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          22297 non-null  object 
 1   PART_NAME            22292 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         15143 non-null  Int64  
 4   BOX_NUMBER           22297 non-null  object 
 5   BOX_TYPE             22297 non-null  object 
 6   BOX_LENGTH_MM        15337 non-null  Int64  
 7   BOX_WIDTH_MM         15337 non-null  Int64  
 8   BOX_HEIGHT_MM        15337 non-null  Int64  
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       15146 non-null  Int64  
 13  LINE_CODE            22297 non-null  object 
 14  LINE_NAME            22297 non-null  object 
 15  MODEL_NAME           22297 non-null 

In [15]:
# удаляем пробелы в начале и в конце значений в колонках PART_NUMBER и SUPPLIER_NAME
mft_source_df[['PART_NUMBER', 'PART_NAME', 'SUPPLIER_NAME']] = mft_source_df[['PART_NUMBER', 'PART_NAME', 'SUPPLIER_NAME']].apply(lambda x: x.str.strip())

# выводим первые строки датафрейма
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CONFIGURATION,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,"Bazhou huihang auto parts co.,Ltd",nan,nan,nan,nan,nan,nan,nan
1,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,"Bazhou huihang auto parts co.,Ltd",nan,nan,nan,nan,nan,nan,nan
2,4004103XKN61A,Жгут проводов салона автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,"Mande Electronics Co., Ltd. Baoding Xushui ele...",nan,nan,nan,nan,nan,nan,nan
3,4004103XKN61A,Жгут проводов салона автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,"Mande Electronics Co., Ltd. Baoding Xushui ele...",nan,nan,nan,nan,nan,nan,nan
4,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,"Bazhou huihang auto parts co.,Ltd",nan,nan,nan,nan,nan,nan,nan


In [16]:
# в колонке PART_NAME разделяем слова с помощью знака '_' в строках, которые написаны стилем "CamelCase"
mft_source_df['PART_NAME'] = mft_source_df['PART_NAME'].apply(
    lambda x: '_'.join(re.findall(r'[А-Я][^А-Я]*', str(x))) if pd.notna(x) else None
)

# выводим первые строки датафрейма
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CONFIGURATION,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,"Bazhou huihang auto parts co.,Ltd",nan,nan,nan,nan,nan,nan,nan
1,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,"Bazhou huihang auto parts co.,Ltd",nan,nan,nan,nan,nan,nan,nan
2,4004103XKN61A,Жгут проводов салона автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,"Mande Electronics Co., Ltd. Baoding Xushui ele...",nan,nan,nan,nan,nan,nan,nan
3,4004103XKN61A,Жгут проводов салона автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,"Mande Electronics Co., Ltd. Baoding Xushui ele...",nan,nan,nan,nan,nan,nan,nan
4,5174104XGW01A,Ниж дефлектор передн бампера в сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,"Bazhou huihang auto parts co.,Ltd",nan,nan,nan,nan,nan,nan,nan


In [17]:
# функция конвертирует китайские иероглифы в латиницу
def convert_to_latin(row):
    return ''.join(lazy_pinyin(row))

In [18]:
# конвертируем китайские иероглифы в латиницу в колонках SUPPLIER_NAME и PART_NAME
mft_source_df[['SUPPLIER_NAME', 'PART_NAME']] = mft_source_df[['SUPPLIER_NAME', 'PART_NAME']].apply(
    lambda x: x.apply(lambda y: convert_to_latin(y) if isinstance(y, str) else y)
)

for col in ['SUPPLIER_NAME', 'PART_NAME']:
    # заменяем комбинацию ".," на "_"
    mft_source_df[col] = mft_source_df[col].str.replace('.,', '_', regex=False)

    # удаляем точки и запятые
    mft_source_df[col] = mft_source_df[col].str.replace(r'[.,]', '', regex=True)

    # заменяем пробелы знаком "_"
    mft_source_df[col] = mft_source_df[col].str.replace(' ', '_', regex=False)

    # двойной знак "__" заменяем одинарным "_"
    mft_source_df[col] = mft_source_df[col].str.replace('__', '_', regex=False)

    # удаляем символы переноса строки
    mft_source_df[col] = mft_source_df[col].str.replace('\n', '', regex=False)

    # приводим все значения к нижнему регистру
    mft_source_df[col] = mft_source_df[col].str.lower()

# выводим первые строки датафрейма
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CONFIGURATION,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,nan,nan,nan
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,nan,nan,nan
2,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,nan,nan,nan
3,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,nan,nan,nan
4,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,nan,nan,nan


In [19]:
# заменяем строки, содержащих подстроку "ecoall" на 'ecoalliance'
mft_source_df.loc[mft_source_df['SUPPLIER_NAME'].str.contains('ecoall'), 'SUPPLIER_NAME'] = 'ecoalliance'

# выводим результат
mft_source_df[mft_source_df['SUPPLIER_NAME'].str.contains('ecoall', case=False)]['SUPPLIER_NAME'].unique()

array(['ecoalliance'], dtype=object)

In [20]:
# заменяем строки, содержащих подстроку "pure" на 'purem_togliatti'
mft_source_df.loc[mft_source_df['SUPPLIER_NAME'].str.contains('pure'), 'SUPPLIER_NAME'] = 'purem_togliatti'

# выводим результат
mft_source_df[mft_source_df['SUPPLIER_NAME'].str.contains('pure', case=False)]['SUPPLIER_NAME'].unique()

array(['purem_togliatti'], dtype=object)

In [21]:
# заменяем строки, содержащих подстроку "smet" на 'resmetal'
mft_source_df.loc[mft_source_df['SUPPLIER_NAME'].str.contains('smet'), 'SUPPLIER_NAME'] = 'resmetal'

# выводим результат
mft_source_df[mft_source_df['SUPPLIER_NAME'].str.contains('smet', case=False)]['SUPPLIER_NAME'].unique()

array(['resmetal'], dtype=object)

In [22]:
# заменяем строки, содержащих подстроку "lukoi" на 'lukoil'
mft_source_df.loc[mft_source_df['SUPPLIER_NAME'].str.contains('lukoi'), 'SUPPLIER_NAME'] = 'lukoil'

# выводим результат
mft_source_df[mft_source_df['SUPPLIER_NAME'].str.contains('lukoi', case=False)]['SUPPLIER_NAME'].unique()

array(['lukoil'], dtype=object)

In [23]:
# выбыраем только локальных поставщиков
local_supplier = [
    'ndkt',
    'fuyao',
    'lear',
    'yapp',
    'gsk',
    'stp',
    'aat',
    'autotechnika',
    'msa',
    'purem_togliatti',
    'knauf',
    'akom',
    'itelma',
    'ecoalliance',
    'litum',
    'resmetal',
    'ecokhim',
    'eftec',
    'lukoil',
    'ruspromholod',
    'sgk',
    'vm_auto',
    'uralelastotehnika',
    'ruichuang'
]

# добавляем колонку LOCALIZATION и ставим значение "yes" для строк,
# где название поставщика совпадает с теми, что в списке, 
# и "no" для тех, где не совпадает
mft_source_df['LOCALIZATION'] = mft_source_df['SUPPLIER_NAME'].apply(lambda x: 'yes' if x in local_supplier else 'no')

# выводим первые строки датафрейма
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CONFIGURATION,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan
2,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,nan
3,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,NaN,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,nan
4,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,NaN,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan


In [24]:
# выводим первые строки датафрейма
initial_line_df.head()

,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME
0,HA0001,Virtual Line,Assembly
1,HAA100,Trim I,Assembly
2,HAA101,Trim I,Assembly
3,HAA102,Trim I,Assembly
4,HAA103,Trim I,Assembly


In [28]:
# выводим первые строки датафрейма
initial_bom_df.head()

,MODEL_CODE,PART_NUMBER,LINE_CODE
0,A01,Q1460616FDE,HAA104
1,A01,09810123,HAA117
2,A01,1607011XKR02A,HAA104
3,A01,Q673B135FDE,HAA106
4,A01,09140323,HAC103


In [33]:
# выводим первые строки датафрейма
initial_workshop_df.head()

,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,Stamping,STAMP
1,Welding,WELD
2,Paint,PAINT
3,Component,COMP
4,Assembly,AS


In [34]:
# объединеняем mft_source_df с initial_bom_df по колонке PART_NUMBER
mft_source_df = mft_source_df.merge(
    initial_bom_df[['PART_NUMBER', 'LINE_CODE']],
    on='PART_NUMBER',
    how='left'
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME,LINE_CODE_y
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan,HAB117
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan,HAB117
2,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,nan,HAA107
3,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,nan,HAA107
4,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan,HAB117


In [35]:
# удаялем дублирующуюся колонку 
mft_source_df.drop(columns='LINE_CODE_x', axis=1, inplace=True)

# переименовываем колонку 
mft_source_df.rename(columns={
    'LINE_CODE_y': 'LINE_CODE'
    }, inplace=True
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,PART_per_VEHICLE,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME,WORKSHOP_short_NAME,LINE_CODE
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan,HAB117
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan,HAB117
2,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,nan,HAA107
3,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,<NA>,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,nan,HAA107
4,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,<NA>,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,nan,HAB117


In [36]:
mft_source_df.drop_duplicates(inplace=True)
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12472 entries, 0 to 144093
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          12472 non-null  object 
 1   PART_NAME            12467 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         10580 non-null  Int64  
 4   BOX_NUMBER           12472 non-null  object 
 5   BOX_TYPE             12472 non-null  object 
 6   BOX_LENGTH_MM        10786 non-null  Int64  
 7   BOX_WIDTH_MM         10786 non-null  Int64  
 8   BOX_HEIGHT_MM        10786 non-null  Int64  
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       10582 non-null  Int64  
 13  LINE_NAME            12472 non-null  object 
 14  MODEL_NAME           12472 non-null  object 
 15  MODEL_CODE           12472 non-null  obj

In [37]:
# объединеняем mft_source_df с initial_line_df по колонке LINE_CODE
mft_source_df = mft_source_df.merge(
    initial_line_df[['LINE_CODE', 'LINE_NAME', 'WORKSHOP_full_NAME']],
    on='LINE_CODE',
    how='left'
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_full_NAME_x,WORKSHOP_short_NAME,LINE_CODE,LINE_NAME_y,WORKSHOP_full_NAME_y
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,nan,nan,HAB117,Chassis I,Assembly
1,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,nan,nan,nan,nan,no,nan,nan,HAA107,Trim I,Assembly
2,8201300XKN02A,верх_защ_кожух_монт_креп_салон_зеркала_зад_вида,NaN,40,nan,nan,365,275,260,NaN,...,nan,nan,nan,nan,no,nan,nan,HAC205,Final II,Assembly
3,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,nan,nan,nan,nan,no,nan,nan,HAD211,Door,Assembly
4,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,nan,nan,nan,nan,no,nan,nan,HAD206,Door,Assembly


In [38]:
# удаялем дублирующиеся колонки
mft_source_df.drop(columns=[
    'LINE_NAME_x',
    'WORKSHOP_full_NAME_x'
    ], axis=1, inplace=True
)

# переименовываем колонки
mft_source_df.rename(columns={
    'LINE_NAME_y': 'LINE_NAME',
    'WORKSHOP_full_NAME_y': 'WORKSHOP_full_NAME'
    }, inplace=True
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_short_NAME,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,nan,HAB117,Chassis I,Assembly
1,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,nan,HAA107,Trim I,Assembly
2,8201300XKN02A,верх_защ_кожух_монт_креп_салон_зеркала_зад_вида,NaN,40,nan,nan,365,275,260,NaN,...,tianjin_hua_tao_auto_plastic_accessories_co_ltd,nan,nan,nan,nan,no,nan,HAC205,Final II,Assembly
3,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,tianjin_minxin_machinery_co_ltd,nan,nan,nan,nan,no,nan,HAD211,Door,Assembly
4,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,tianjin_minxin_machinery_co_ltd,nan,nan,nan,nan,no,nan,HAD206,Door,Assembly


In [39]:
mft_source_df.drop_duplicates(inplace=True)
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12472 entries, 0 to 12471
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          12472 non-null  object 
 1   PART_NAME            12467 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         10580 non-null  Int64  
 4   BOX_NUMBER           12472 non-null  object 
 5   BOX_TYPE             12472 non-null  object 
 6   BOX_LENGTH_MM        10786 non-null  Int64  
 7   BOX_WIDTH_MM         10786 non-null  Int64  
 8   BOX_HEIGHT_MM        10786 non-null  Int64  
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       10582 non-null  Int64  
 13  MODEL_NAME           12472 non-null  object 
 14  MODEL_CODE           12472 non-null  object 
 15  PALLET_NUMBER        12472 non-null 

In [40]:
# объединеняем mft_source_df с initial_workshop_df по колонке WORKSHOP_full_NAME
mft_source_df = mft_source_df.merge(
    initial_workshop_df[['WORKSHOP_full_NAME', 'WORKSHOP_short_NAME']],
    on='WORKSHOP_full_NAME',
    how='left'
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,WORKSHOP_short_NAME_x,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME_y
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,nan,HAB117,Chassis I,Assembly,AS
1,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,nan,nan,nan,nan,no,nan,HAA107,Trim I,Assembly,AS
2,8201300XKN02A,верх_защ_кожух_монт_креп_салон_зеркала_зад_вида,NaN,40,nan,nan,365,275,260,NaN,...,nan,nan,nan,nan,no,nan,HAC205,Final II,Assembly,AS
3,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,nan,nan,nan,nan,no,nan,HAD211,Door,Assembly,AS
4,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,nan,nan,nan,nan,no,nan,HAD206,Door,Assembly,AS


In [41]:
# удаялем дублирующиеся колонки
mft_source_df.drop(columns=[
    'WORKSHOP_short_NAME_x'
    ], axis=1, inplace=True
)

# переименовываем колонки
mft_source_df.rename(columns={
    'WORKSHOP_short_NAME_y': 'WORKSHOP_short_NAME'
    }, inplace=True
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,SUPPLIER_NAME,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,bazhou_huihang_auto_parts_co_ltd,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS
1,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,mande_electronics_co_ltd_baoding_xushui_electr...,nan,nan,nan,nan,no,HAA107,Trim I,Assembly,AS
2,8201300XKN02A,верх_защ_кожух_монт_креп_салон_зеркала_зад_вида,NaN,40,nan,nan,365,275,260,NaN,...,tianjin_hua_tao_auto_plastic_accessories_co_ltd,nan,nan,nan,nan,no,HAC205,Final II,Assembly,AS
3,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,tianjin_minxin_machinery_co_ltd,nan,nan,nan,nan,no,HAD211,Door,Assembly,AS
4,6213120XKN02A,панель_обив_пер_рамы_стекла_пр_зад_двери_в_сборе,NaN,20,nan,nan,480,365,260,NaN,...,tianjin_minxin_machinery_co_ltd,nan,nan,nan,nan,no,HAD206,Door,Assembly,AS


In [42]:
mft_source_df.drop_duplicates(inplace=True)
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12472 entries, 0 to 12471
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          12472 non-null  object 
 1   PART_NAME            12467 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         10580 non-null  Int64  
 4   BOX_NUMBER           12472 non-null  object 
 5   BOX_TYPE             12472 non-null  object 
 6   BOX_LENGTH_MM        10786 non-null  Int64  
 7   BOX_WIDTH_MM         10786 non-null  Int64  
 8   BOX_HEIGHT_MM        10786 non-null  Int64  
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       10582 non-null  Int64  
 13  MODEL_NAME           12472 non-null  object 
 14  MODEL_CODE           12472 non-null  object 
 15  PALLET_NUMBER        12472 non-null 

In [43]:
# выбираем нужные колонки для создания рабочего датафрейма
initial_part_to_model_df = mft_rough_df[
    [
        'партномер',
        'A01',
        'A08',
        'B02',
        'B04',
        'B06/B06X',
        'B16'
    ]
].copy()

# выводим первые строки датафрейма
initial_part_to_model_df.head()

,партномер,A01,A08,B02,B04,B06/B06X,B16
0,5174104XGW01A,NaN,NaN,+,+,+,+
1,5174104XGW01A,NaN,NaN,+,+,+,+
2,4004103XKN61A,NaN,NaN,+,NaN,NaN,NaN
3,4004103XKN61A,NaN,NaN,+,NaN,NaN,NaN
4,5174104XGW01A,NaN,NaN,+,+,+,+


In [44]:
# переименовываем названия колонок
initial_part_to_model_df.rename(
    columns={
        'партномер': 'PART_NUMBER',
        'B06/B06X':'B06',
    }, inplace=True
)

# выводим первые строки датафрейма
initial_part_to_model_df.head()

,PART_NUMBER,A01,A08,B02,B04,B06,B16
0,5174104XGW01A,NaN,NaN,+,+,+,+
1,5174104XGW01A,NaN,NaN,+,+,+,+
2,4004103XKN61A,NaN,NaN,+,NaN,NaN,NaN
3,4004103XKN61A,NaN,NaN,+,NaN,NaN,NaN
4,5174104XGW01A,NaN,NaN,+,+,+,+


In [45]:
# во всех колонках датафрейма удаляем пробелы в начале и в конце строк
for col in initial_part_to_model_df.columns:
    if initial_part_to_model_df[col].dtype == object:
        initial_part_to_model_df[col] = initial_part_to_model_df[col].astype(str).str.strip().replace('', np.nan)

# выводим результат
initial_part_to_model_df.head(10)

,PART_NUMBER,A01,A08,B02,B04,B06,B16
0,5174104XGW01A,nan,nan,+,+,+,+
1,5174104XGW01A,nan,nan,+,+,+,+
2,4004103XKN61A,nan,nan,+,nan,nan,nan
3,4004103XKN61A,nan,nan,+,nan,nan,nan
4,5174104XGW01A,nan,nan,+,+,+,+
5,5174104XGW01A,nan,nan,+,+,+,+
6,8201300XKN02A,nan,nan,+,+,nan,nan
7,6213120XKN02A,nan,nan,+,+,nan,nan
8,4013102XKN02B,nan,nan,+,nan,nan,nan
9,4121124XKN02A,nan,nan,+,nan,nan,nan


In [46]:
# проверяем на наличие дубликатов
initial_part_to_model_df[initial_part_to_model_df.duplicated()]

,PART_NUMBER,A01,A08,B02,B04,B06,B16
1,5174104XGW01A,nan,nan,+,+,+,+
3,4004103XKN61A,nan,nan,+,nan,nan,nan
4,5174104XGW01A,nan,nan,+,+,+,+
5,5174104XGW01A,nan,nan,+,+,+,+
27,3603210XKZ4JA9C,nan,nan,+,nan,nan,nan
...,...,...,...,...,...,...,...
21934,5604101AKN02A,nan,nan,+,+,nan,nan
22090,7002230XST33A8P,+,nan,nan,nan,nan,nan
22175,1205112XST11A,+,+,+,+,+,+
22176,1205113XST11A,+,+,+,+,+,+


In [47]:
# удаляем дубликаты
initial_part_to_model_df = initial_part_to_model_df.drop_duplicates(inplace=False)

# проверяем на наличие дубликатов
initial_part_to_model_df[initial_part_to_model_df.duplicated()]

,PART_NUMBER,A01,A08,B02,B04,B06,B16


In [48]:
# меняем структуру датафрейма final_part_to_model_df
# список столбцов с названиями моделей
model_cols = ['A01', 'A08', 'B02', 'B04', 'B06', 'B16']

# развертываем таблицу, создавая одну строку на каждую модель, где есть '+'
melted_df = initial_part_to_model_df.melt(
    id_vars=['PART_NUMBER'],
    value_vars=model_cols,
    var_name='MODEL_CODE',
    value_name='USED_IN_MODEL'
)

# оставляем только те записи, где используется деталь ('+' в старой таблице)
melted_df = melted_df[melted_df['USED_IN_MODEL'] == '+']

# удаляем ненужный столбец USED_IN_MODEL
melted_df.drop('USED_IN_MODEL', axis=1, inplace=True)

# получаем итоговую версию датафрейма
final_part_to_model_df = melted_df.reset_index(drop=True)

# выводим первые строки датафрейма
final_part_to_model_df.head()

,PART_NUMBER,MODEL_CODE
0,1203017-V08,A01
1,1119120XGW02A,A01
2,1205012XKZ3FA,A01
3,4116300XKN01A,A01
4,09220016,A01


In [49]:
# объединеняем mft_source_df с initial_workshop_df по колонке PART_NUMBER
mft_source_df = mft_source_df.merge(
    final_part_to_model_df[['PART_NUMBER', 'MODEL_CODE']],
    on='PART_NUMBER',
    how='left'
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME,MODEL_CODE_y
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B02
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B04
2,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B06
3,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B16
4,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,nan,nan,nan,nan,no,HAA107,Trim I,Assembly,AS,B02


In [50]:
# удаялем дублирующиеся колонки
mft_source_df.drop(columns=[
    'MODEL_CODE_x'
    ], axis=1, inplace=True
)

# переименовываем колонки
mft_source_df.rename(columns={
    'MODEL_CODE_y': 'MODEL_CODE'
    }, inplace=True
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,LOCATION,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME,MODEL_CODE
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B02
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B04
2,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B06
3,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B16
4,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,nan,nan,nan,nan,no,HAA107,Trim I,Assembly,AS,B02


In [51]:
mft_source_df.drop_duplicates(inplace=True)
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40681 entries, 0 to 49423
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        37425 non-null  Int64  
 7   BOX_WIDTH_MM         37425 non-null  Int64  
 8   BOX_HEIGHT_MM        37425 non-null  Int64  
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       37099 non-null  Int64  
 13  MODEL_NAME           40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null  obje

In [52]:
# объединеняем mft_source_df с initial_workshop_df по колонке PART_NUMBER
mft_source_df = mft_source_df.merge(
    initial_model_df[['MODEL_CODE', 'MODEL_NAME']],
    on='MODEL_CODE',
    how='left'
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME,MODEL_CODE,MODEL_NAME_y
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B02,F7
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B04,F7x
2,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B06,Dargo
3,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,nan,nan,1340,560,440,NaN,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B16,H7
4,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,nan,nan,550,480,260,NaN,...,nan,nan,nan,no,HAA107,Trim I,Assembly,AS,B02,F7


In [64]:
# удаялем дублирующиеся колонки
mft_source_df.drop(columns=[
    'MODEL_NAME_x'
    ], axis=1, inplace=True
)

# переименовываем колонки
mft_source_df.rename(columns={
    'MODEL_NAME_y': 'MODEL_NAME'
    }, inplace=True
)

# выводим результирующий датафрейм
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME,MODEL_CODE,MODEL_NAME
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B02,F7
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B04,F7x
2,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B06,Dargo
3,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B16,H7
4,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,A 550-480-260,n_returnable,550,480,260,0.07,...,nan,nan,nan,no,HAA107,Trim I,Assembly,AS,B02,F7


In [65]:
mft_source_df.drop_duplicates(inplace=True)
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          38809 non-null  Float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  PALLET_NUMBER        40681 non-null  object 
 14  PALLET_TYPE          40681 non-null  object 
 15  PALLET_LENGTH_MM     4147 non-null  

In [54]:
# если в колонках BOX_*_MM нет данных 
# копируем в них данные из колонок PALLET_*_MM

# маска для проверки наличия пустых значений в колонках BOX_*_MM
mask = (
    (mft_source_df['BOX_LENGTH_MM'].isna()) &
    (mft_source_df['BOX_WIDTH_MM'].isna()) &
    (mft_source_df['BOX_HEIGHT_MM'].isna())
)

# копируем значения из PALLET_*_MM в BOX_*_MM там, где условие выполняется
mft_source_df.loc[mask, 'BOX_LENGTH_MM'] = mft_source_df.loc[mask, 'PALLET_LENGTH_MM']
mft_source_df.loc[mask, 'BOX_WIDTH_MM'] = mft_source_df.loc[mask, 'PALLET_WIDTH_MM']
mft_source_df.loc[mask, 'BOX_HEIGHT_MM'] = mft_source_df.loc[mask, 'PALLET_HEIGHT_MM']

# устанавливаем значение 1 в колонку BOX_per_PALLET,
# так как предполагается, что на 1 паллету ставится 1 ящик
mft_source_df.loc[mask, 'BOX_per_PALLET'] = 1

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           0 non-null      float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [55]:
# пересчитываем объем коробок в датафрейме box_df
mft_source_df['BOX_VOL_M3'] = round(
    (mft_source_df['BOX_LENGTH_MM'] / 1000) * 
    (mft_source_df['BOX_WIDTH_MM'] / 1000) * 
    (mft_source_df['BOX_HEIGHT_MM'] / 1000), 2)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          0 non-null      float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [56]:
# пересчитываем площадь коробок в датафрейме box_df
mft_source_df['BOX_AREA_M2'] = round(
    (mft_source_df['BOX_LENGTH_MM'] / 1000) * 
    (mft_source_df['BOX_WIDTH_MM'] / 1000), 2)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          38809 non-null  Float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [57]:
# пересчитываем объем паллет в датафрейме box_df
mft_source_df['PALLET_VOL_M3'] = round(
    (mft_source_df['PALLET_LENGTH_MM'] / 1000) * 
    (mft_source_df['PALLET_WIDTH_MM'] / 1000) * 
    (mft_source_df['PALLET_HEIGHT_MM'] / 1000), 2)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          38809 non-null  Float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [58]:
# пересчитываем площадь паллет в датафрейме box_df
mft_source_df['PALLET_AREA_M2'] = round(
    (mft_source_df['PALLET_LENGTH_MM'] / 1000) * 
    (mft_source_df['PALLET_WIDTH_MM'] / 1000), 2)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          38809 non-null  Float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [59]:
# создаем в колонке BOX_NUMBER номера коробок,
# сцепляя значения из колонок BOX_LENGTH_MM, BOX_WIDTH_MM, BOX_HEIGHT_MM,
# присваивая в начале номера литеру "А" - "невозвратная тара"

mft_source_df['BOX_NUMBER'] = mft_source_df.apply(
    lambda row: f"A {row['BOX_LENGTH_MM']}-{row['BOX_WIDTH_MM']}-{row['BOX_HEIGHT_MM']}",
    axis=1
)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          38809 non-null  Float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [60]:
# создаем в колонке PALLET_NUMBER номера коробок,
# сцепляя значения из колонок PALLET_LENGTH_MM, PALLET_WIDTH_MM, PALLET_HEIGHT_MM,
# присваивая в начале номера литеру "А" - "невозвратная тара"

mft_source_df['PALLET_NUMBER'] = mft_source_df.apply(
    lambda row: f"A {row['PALLET_LENGTH_MM']}-{row['PALLET_WIDTH_MM']}-{row['PALLET_HEIGHT_MM']}",
    axis=1
)

# выводим информацию о датафрейме
mft_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40681 entries, 0 to 40680
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PART_NUMBER          40681 non-null  object 
 1   PART_NAME            40674 non-null  object 
 2   PART_WEIGHT_KG       0 non-null      float64
 3   PART_per_BOX         37097 non-null  Int64  
 4   BOX_NUMBER           40681 non-null  object 
 5   BOX_TYPE             40681 non-null  object 
 6   BOX_LENGTH_MM        38809 non-null  Int64  
 7   BOX_WIDTH_MM         38809 non-null  Int64  
 8   BOX_HEIGHT_MM        38809 non-null  Int64  
 9   BOX_VOL_M3           38809 non-null  Float64
 10  BOX_AREA_M2          38809 non-null  Float64
 11  BOX_STACKING         0 non-null      Int64  
 12  BOX_per_PALLET       40349 non-null  Int64  
 13  MODEL_NAME_x         40681 non-null  object 
 14  PALLET_NUMBER        40681 non-null  object 
 15  PALLET_TYPE          40681 non-null 

In [61]:
# вносим значение n_returnable (невозвратная) в колонки BOX_TYPE и PALLET_TYPE
mft_source_df[['BOX_TYPE', 'PALLET_TYPE']] = 'n_returnable'

# выводим первые строки датафрейма
mft_source_df.head()

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME,MODEL_CODE,MODEL_NAME_y
0,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B02,F7
1,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B04,F7x
2,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B06,Dargo
3,5174104XGW01A,ниж_дефлектор_передн_бампера_в_сборе,NaN,5,A 1340-560-440,n_returnable,1340,560,440,0.33,...,nan,nan,nan,no,HAB117,Chassis I,Assembly,AS,B16,H7
4,4004103XKN61A,жгут_проводов_салона_автомобиля,NaN,1,A 550-480-260,n_returnable,550,480,260,0.07,...,nan,nan,nan,no,HAA107,Trim I,Assembly,AS,B02,F7


In [62]:
mft_source_df[mft_source_df.duplicated()]

,PART_NUMBER,PART_NAME,PART_WEIGHT_KG,PART_per_BOX,BOX_NUMBER,BOX_TYPE,BOX_LENGTH_MM,BOX_WIDTH_MM,BOX_HEIGHT_MM,BOX_VOL_M3,...,CITY,STREET,BUILDING,LOCALIZATION,LINE_CODE,LINE_NAME,WORKSHOP_full_NAME,WORKSHOP_short_NAME,MODEL_CODE,MODEL_NAME_y


In [58]:
# сохраняем заполненный файл с данными
mft_source_df.to_excel(r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\source\mft_source_full.xlsx', index=False)

# Технические изменения

In [76]:
# создаем датафрейм с сырыми данными
breakpoint_df = pd.read_excel(breakpoint_path)

# выводим первые строки датафрейма
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION
0,NaN,1201562XGW02A,1201562XGW02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,6802300XST32AQT,6802300XST33AQT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,6802300XST32AZ6,6802300XST33AZ6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,6802700XST01AQT,6802700XST33AQT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,6802700XST01AZ6,6802700XST33AZ6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# создаем датафрейм с сырыми данными
bp_list_df = pd.read_excel(bp_list_path)

# выводим первые строки датафрейма
bp_list_df.head()

,BP,IsUnBomBP,Status,Batch in BP (Plan),Start Batch (Fact),Date,Model,Change,Operation,Part number,Part Name (C),Part Name (E),Workshop,Comments
0,BP24004128,No,Closed,RAR0214,RAR0214,2025-05-06 00:00:00,A01G-R,NaN,Add,5130807XST11A,后地板前横梁加强板,"PLATE,RR FLOOR FR CROSS MBR REINF",WELD,NaN
1,BP24004128,No,Closed,RAR0214,RAR0214,2025-05-06 00:00:00,A01G-R,NaN,Add,5130477XST11A,后地板前横梁左加强板,"PLATE,RR FLOOR FR CROSS MBR REINF,LH",WELD,NaN
2,BP24004128,No,Closed,RAR0214,RAR0214,2025-05-06 00:00:00,A01G-R,NaN,Add,5130397XST11A,后地板前横梁右加强板,"PLATE,RR FLOOR FR CROSS MBR REINF,RH",WELD,NaN
3,BP24004128,No,Closed,RAR0214,RAR0214,2025-05-06 00:00:00,A01G-R,NaN,Add,5130883XST11A,后地板左纵梁前段加强板,"PLATE,LH MAIN SILL REINF,FR",WELD,NaN
4,BP24004128,No,Closed,RAR0214,RAR0214,2025-05-06 00:00:00,A01G-R,NaN,Add,5130884XST11A,后地板右纵梁前段加强板,"PLATE,RH MAIN SILL FR SECTION REINF",WELD,NaN


In [79]:
# Объединяем два датафрейма, выбирая нужные столбцы и устанавливаем соединение на основе совпадения номеров деталей
merged_df = breakpoint_df.merge(bp_list_df, left_on='OLD_PART_NUMBER', right_on='Part number', how='left')

# Обновляем BREAKPOINT_NUMBER значениями BP там, где есть совпадение
breakpoint_df['BREAKPOINT_NUMBER'] = merged_df['BP']

# Выыодим непустые ячейки
breakpoint_df[['OLD_PART_NUMBER', 'NEW_PART_NUMBER', 'BREAKPOINT_NUMBER']].dropna(subset=['BREAKPOINT_NUMBER'])

,OLD_PART_NUMBER,NEW_PART_NUMBER,BREAKPOINT_NUMBER
0,1201562XGW02A,1201562XGW02B,BP24011790
1,6802300XST32AQT,6802300XST33AQT,BP24011789
2,6802300XST32AZ6,6802300XST33AZ6,BP24011789
3,6802700XST01AQT,6802700XST33AQT,BP24011789
4,6802700XST01AZ6,6802700XST33AZ6,BP24011789
...,...,...,...
98,7000101XKN81A8P,7000105XKN81A8P,BP25000876
99,6203100XKN08A,6203100XKN83A,BP25002734
100,6203200XKN08A,6203200XKN83A,BP25002734
101,7000103XKN81AC0F,7000107XKN81AC0F,BP25003498


In [80]:
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION
0,NaN,1201562XGW02A,1201562XGW02B,NaN,BP24011790,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,6802300XST32AQT,6802300XST33AQT,NaN,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,6802300XST32AZ6,6802300XST33AZ6,NaN,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,6802700XST01AQT,6802700XST33AQT,NaN,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,6802700XST01AZ6,6802700XST33AZ6,NaN,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Объединяем два датафрейма, выбирая нужные столбцы и устанавливаем соединение на основе совпадения номеров деталей
merged_df = breakpoint_df.merge(bp_list_df, left_on='OLD_PART_NUMBER', right_on='Part number', how='left')

# Обновляем PART_NAME значениями Part Name (E) там, где есть совпадение
breakpoint_df['PART_NAME'] = merged_df['Part Name (C)']

# Выыодим непустые ячейки
breakpoint_df[['OLD_PART_NUMBER', 'NEW_PART_NUMBER', 'PART_NAME']].dropna(subset=['PART_NAME'])

,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME
0,1201562XGW02A,1201562XGW02B,主消声器分总成
1,6802300XST32AQT,6802300XST33AQT,驾驶员座椅靠背护面总成
2,6802300XST32AZ6,6802300XST33AZ6,驾驶员座椅靠背护面总成
3,6802700XST01AQT,6802700XST33AQT,驾驶员座椅座垫护面总成
4,6802700XST01AZ6,6802700XST33AZ6,驾驶员座椅座垫护面总成
...,...,...,...
98,7000101XKN81A8P,7000105XKN81A8P,行李箱地毯
99,6203100XKN08A,6203100XKN83A,车载无线终端总成
100,6203200XKN08A,6203200XKN83A,车载无线终端总成
101,7000103XKN81AC0F,7000107XKN81AC0F,车载无线终端


In [82]:
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION
0,NaN,1201562XGW02A,1201562XGW02B,主消声器分总成,BP24011790,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,6802300XST32AQT,6802300XST33AQT,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,6802300XST32AZ6,6802300XST33AZ6,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,6802700XST01AQT,6802700XST33AQT,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,6802700XST01AZ6,6802700XST33AZ6,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# Объединяем два датафрейма, выбирая нужные столбцы и устанавливаем соединение на основе совпадения номеров деталей
merged_df = breakpoint_df.merge(bp_list_df, left_on='OLD_PART_NUMBER', right_on='Part number', how='left')

# Обновляем PART_NAME значениями Part Name (E) там, где есть совпадение
breakpoint_df['MODEL_CODE'] = merged_df['Model']

# Выыодим непустые ячейки
breakpoint_df[['OLD_PART_NUMBER', 'NEW_PART_NUMBER', 'MODEL_CODE']].dropna(subset=['MODEL_CODE'])

,OLD_PART_NUMBER,NEW_PART_NUMBER,MODEL_CODE
0,1201562XGW02A,1201562XGW02B,A01G-R
1,6802300XST32AQT,6802300XST33AQT,A01G-R
2,6802300XST32AZ6,6802300XST33AZ6,A01G-R
3,6802700XST01AQT,6802700XST33AQT,A01G-R
4,6802700XST01AZ6,6802700XST33AZ6,A01G-R
...,...,...,...
98,7000101XKN81A8P,7000105XKN81A8P,B06-R
99,6203100XKN08A,6203100XKN83A,B06-R
100,6203200XKN08A,6203200XKN83A,B06-X
101,7000103XKN81AC0F,7000107XKN81AC0F,B16-R


In [84]:
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION
0,NaN,1201562XGW02A,1201562XGW02B,主消声器分总成,BP24011790,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
1,NaN,6802300XST32AQT,6802300XST33AQT,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
2,NaN,6802300XST32AZ6,6802300XST33AZ6,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
3,NaN,6802700XST01AQT,6802700XST33AQT,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
4,NaN,6802700XST01AZ6,6802700XST33AZ6,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN


In [85]:
# Объединяем два датафрейма, выбирая нужные столбцы и устанавливаем соединение на основе совпадения номеров деталей
merged_df = breakpoint_df.merge(bp_list_df, left_on='BREAKPOINT_NUMBER', right_on='BP', how='left')

# Обновляем PART_NAME значениями Part Name (E) там, где есть совпадение
breakpoint_df['DATE'] = merged_df['Date']

# Выыодим непустые ячейки
breakpoint_df[['DATE', 'BREAKPOINT_NUMBER']].dropna(subset=['BREAKPOINT_NUMBER'])

,DATE,BREAKPOINT_NUMBER
0,2025-05-16 00:00:00,BP24011790
1,2025-05-16 00:00:00,BP24011789
2,2025-05-16 00:00:00,BP24011789
3,2025-05-16 00:00:00,BP24011789
4,2025-03-04 00:00:00,BP24011789
...,...,...
98,2025-03-04 00:00:00,BP25000876
99,2025-03-04 00:00:00,BP25002734
100,2025-03-04 00:00:00,BP25002734
101,2025-03-04 00:00:00,BP25003498


In [86]:
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION
0,2025-05-16 00:00:00,1201562XGW02A,1201562XGW02B,主消声器分总成,BP24011790,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
1,2025-05-16 00:00:00,6802300XST32AQT,6802300XST33AQT,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
2,2025-05-16 00:00:00,6802300XST32AZ6,6802300XST33AZ6,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
3,2025-05-16 00:00:00,6802700XST01AQT,6802700XST33AQT,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN
4,2025-03-04 00:00:00,6802700XST01AZ6,6802700XST33AZ6,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN


In [88]:
# Загружаем JSON-файл
with open('part_number_translate.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Преобразовываем словарь в датафрейм
translate_df = pd.DataFrame(list(data.items()), columns=["PART_NUMBER", "PART_NAME"])

# Выводим первые строки датафрейма
print(translate_df.head())

       PART_NUMBER                                       PART_NAME
0    1201562XGW02A               Основной глушитель (сборный узел)
1  6802300XST32AQT   Обивка спинки сиденья водителя (сборный узел)
2  6802300XST32AZ6   Обивка спинки сиденья водителя (сборный узел)
3  6802700XST01AQT  Обивка подушки сиденья водителя (сборный узел)
4  6802700XST01AZ6  Обивка подушки сиденья водителя (сборный узел)


In [89]:
# Обычная замена строки
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace('(сборный узел)', '', regex=False)
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace('(сборочный узел)', '', regex=False)

# Удаляем пробелы справа и слева, если появились
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.strip()

# заменяем комбинацию ".," на "_"
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace('.,', '_', regex=False)

# удаляем точки и запятые
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace(r'[.,]', '', regex=True)

# удаляем круглые скобки
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace(r'[)(]', '', regex=True)

# заменяем пробелы знаком "_"
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace(' ', '_', regex=False)

# двойной знак "__" заменяем одинарным "_"
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace('__', '_', regex=False)

# удаляем символы переноса строки
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.replace('\n', '', regex=False)

# приводим все значения к нижнему регистру
translate_df['PART_NAME'] = translate_df['PART_NAME'].str.lower()

# Вывод результата
translate_df

,PART_NUMBER,PART_NAME
0,1201562XGW02A,основной_глушитель
1,6802300XST32AQT,обивка_спинки_сиденья_водителя
2,6802300XST32AZ6,обивка_спинки_сиденья_водителя
3,6802700XST01AQT,обивка_подушки_сиденья_водителя
4,6802700XST01AZ6,обивка_подушки_сиденья_водителя
...,...,...
74,7000103XKN81A8P,бортовой_беспроводной_терминал
75,7000101XKN81A8P,ковровое_покрытие_багажного_отделения
76,6203100XKN08A,бортовой_беспроводной_терминал
77,6203200XKN08A,бортовой_беспроводной_терминал


In [70]:
# Сохраняем dataframe в файл формата .xlsx
translate_df.to_excel(r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\deep_seek_translate.xlsx', index=False)

In [90]:
# Слияние датафреймов по общему полю
breakpoint_df = breakpoint_df.merge(translate_df, left_on='OLD_PART_NUMBER', right_on='PART_NUMBER', how='left')

# Выводим результат
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME_x,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION,PART_NUMBER,PART_NAME_y
0,2025-05-16 00:00:00,1201562XGW02A,1201562XGW02B,主消声器分总成,BP24011790,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,1201562XGW02A,основной_глушитель
1,2025-05-16 00:00:00,6802300XST32AQT,6802300XST33AQT,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802300XST32AQT,обивка_спинки_сиденья_водителя
2,2025-05-16 00:00:00,6802300XST32AZ6,6802300XST33AZ6,驾驶员座椅靠背护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802300XST32AZ6,обивка_спинки_сиденья_водителя
3,2025-05-16 00:00:00,6802700XST01AQT,6802700XST33AQT,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802700XST01AQT,обивка_подушки_сиденья_водителя
4,2025-03-04 00:00:00,6802700XST01AZ6,6802700XST33AZ6,驾驶员座椅座垫护面总成,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802700XST01AZ6,обивка_подушки_сиденья_водителя


In [91]:
breakpoint_df.columns

Index(['DATE', 'OLD_PART_NUMBER', 'NEW_PART_NUMBER', 'PART_NAME_x',
       'BREAKPOINT_NUMBER', 'OLD_LINE_CODE', 'NEW_LINE_CODE', 'MODEL_CODE',
       'MODEL_NAME', 'OLD_SUPPLIER_NAME', 'NEW_SUPPLIER_NAME', 'LOCALIZATION',
       'PART_NUMBER', 'PART_NAME_y'],
      dtype='object')

In [92]:
breakpoint_df = breakpoint_df.drop(columns=['PART_NAME_x'], axis=1)

breakpoint_df = breakpoint_df.rename(columns={
    'PART_NAME_y': 'PART_NAME'
    }, inplace=False
)

# Выводим результат
breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION,PART_NUMBER,PART_NAME
0,2025-05-16 00:00:00,1201562XGW02A,1201562XGW02B,BP24011790,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,1201562XGW02A,основной_глушитель
1,2025-05-16 00:00:00,6802300XST32AQT,6802300XST33AQT,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802300XST32AQT,обивка_спинки_сиденья_водителя
2,2025-05-16 00:00:00,6802300XST32AZ6,6802300XST33AZ6,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802300XST32AZ6,обивка_спинки_сиденья_водителя
3,2025-05-16 00:00:00,6802700XST01AQT,6802700XST33AQT,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802700XST01AQT,обивка_подушки_сиденья_водителя
4,2025-03-04 00:00:00,6802700XST01AZ6,6802700XST33AZ6,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802700XST01AZ6,обивка_подушки_сиденья_водителя


In [94]:
breakpoint_df = breakpoint_df[
    [
        'DATE',
        'OLD_PART_NUMBER',
        'NEW_PART_NUMBER',
        'PART_NAME',
        'BREAKPOINT_NUMBER',
        'OLD_LINE_CODE',
        'NEW_LINE_CODE',
        'MODEL_CODE',
        'MODEL_NAME',
        'OLD_SUPPLIER_NAME',
        'NEW_SUPPLIER_NAME',
        'LOCALIZATION',
        'PART_NUMBER'
    ]
]

breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION,PART_NUMBER
0,2025-05-16 00:00:00,1201562XGW02A,1201562XGW02B,основной_глушитель,BP24011790,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,1201562XGW02A
1,2025-05-16 00:00:00,6802300XST32AQT,6802300XST33AQT,обивка_спинки_сиденья_водителя,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802300XST32AQT
2,2025-05-16 00:00:00,6802300XST32AZ6,6802300XST33AZ6,обивка_спинки_сиденья_водителя,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802300XST32AZ6
3,2025-05-16 00:00:00,6802700XST01AQT,6802700XST33AQT,обивка_подушки_сиденья_водителя,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802700XST01AQT
4,2025-03-04 00:00:00,6802700XST01AZ6,6802700XST33AZ6,обивка_подушки_сиденья_водителя,BP24011789,NaN,NaN,A01G-R,NaN,NaN,NaN,NaN,6802700XST01AZ6


In [103]:
# Оставляем только первые три символа
breakpoint_df['MODEL_CODE'] = breakpoint_df['MODEL_CODE'].apply(lambda x: x[:3] if isinstance(x, str) else x)

breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION,PART_NUMBER
0,2025-05-16 00:00:00,1201562XGW02A,1201562XGW02B,основной_глушитель,BP24011790,NaN,NaN,A01,NaN,NaN,NaN,NaN,1201562XGW02A
1,2025-05-16 00:00:00,6802300XST32AQT,6802300XST33AQT,обивка_спинки_сиденья_водителя,BP24011789,NaN,NaN,A01,NaN,NaN,NaN,NaN,6802300XST32AQT
2,2025-05-16 00:00:00,6802300XST32AZ6,6802300XST33AZ6,обивка_спинки_сиденья_водителя,BP24011789,NaN,NaN,A01,NaN,NaN,NaN,NaN,6802300XST32AZ6
3,2025-05-16 00:00:00,6802700XST01AQT,6802700XST33AQT,обивка_подушки_сиденья_водителя,BP24011789,NaN,NaN,A01,NaN,NaN,NaN,NaN,6802700XST01AQT
4,2025-03-04 00:00:00,6802700XST01AZ6,6802700XST33AZ6,обивка_подушки_сиденья_водителя,BP24011789,NaN,NaN,A01,NaN,NaN,NaN,NaN,6802700XST01AZ6


In [99]:
initial_model_df.columns

Index(['MODEL_CODE', 'MODEL_NAME'], dtype='object')

In [104]:
# Объединяем датафреймы по ключевому полю MODEL_CODE
merged_df = breakpoint_df.merge(initial_model_df, on='MODEL_CODE', how='left', suffixes=('_original', ''))

# Обновляем столбец MODEL_NAME из объединённой таблицы
breakpoint_df['MODEL_NAME'] = merged_df['MODEL_NAME']

breakpoint_df.head()

,DATE,OLD_PART_NUMBER,NEW_PART_NUMBER,PART_NAME,BREAKPOINT_NUMBER,OLD_LINE_CODE,NEW_LINE_CODE,MODEL_CODE,MODEL_NAME,OLD_SUPPLIER_NAME,NEW_SUPPLIER_NAME,LOCALIZATION,PART_NUMBER
0,2025-05-16 00:00:00,1201562XGW02A,1201562XGW02B,основной_глушитель,BP24011790,NaN,NaN,A01,Jolion,NaN,NaN,NaN,1201562XGW02A
1,2025-05-16 00:00:00,6802300XST32AQT,6802300XST33AQT,обивка_спинки_сиденья_водителя,BP24011789,NaN,NaN,A01,Jolion,NaN,NaN,NaN,6802300XST32AQT
2,2025-05-16 00:00:00,6802300XST32AZ6,6802300XST33AZ6,обивка_спинки_сиденья_водителя,BP24011789,NaN,NaN,A01,Jolion,NaN,NaN,NaN,6802300XST32AZ6
3,2025-05-16 00:00:00,6802700XST01AQT,6802700XST33AQT,обивка_подушки_сиденья_водителя,BP24011789,NaN,NaN,A01,Jolion,NaN,NaN,NaN,6802700XST01AQT
4,2025-03-04 00:00:00,6802700XST01AZ6,6802700XST33AZ6,обивка_подушки_сиденья_водителя,BP24011789,NaN,NaN,A01,Jolion,NaN,NaN,NaN,6802700XST01AZ6


In [105]:
# Сохраняем dataframe в файл формата .xlsx
breakpoint_df.to_excel(r'C:\Users\gw07011219\Documents\barikov_vladimir\projects\mft_db\datapreparation\sample_breakpoint_data_full.xlsx', index=False)